Setup & configuration

In [14]:
# === Section 1: Project paths & constants (run this first) ===
from pathlib import Path
import os

# --- Resolve project root robustly (works in notebooks, VS Code, and scripts) ---
if "__file__" in globals():
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
else:
    # If working in a notebook, assume the repo root is one level up from the notebook
    # but allow an env override if you prefer:  os.environ["PROJECT_ROOT"] = "C:/path/to/repo"
    PROJECT_ROOT = Path(os.environ.get("PROJECT_ROOT", "..")).resolve()

print(f"PROJECT_ROOT = {PROJECT_ROOT}")

# --- Standard data directories ---
DATA_RAW       = PROJECT_ROOT / "data" / "raw"
DATA_INTERIM   = PROJECT_ROOT / "data" / "interim"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

for d in (DATA_RAW, DATA_INTERIM, DATA_PROCESSED):
    d.mkdir(parents=True, exist_ok=True)

print(f"Using DATA_INTERIM = {DATA_INTERIM}")
print(f"Using DATA_PROCESSED = {DATA_PROCESSED}")

# --- Expected raw file locations (adjust if your layout differs) ---
files = {
    "k": DATA_RAW / "UKHLS" / "k_indresp.sav",
    "l": DATA_RAW / "UKHLS" / "l_indresp.sav",
    "n": DATA_RAW / "UKHLS" / "n_indresp.sav",
}

# Quick existence check with a friendly hint if something’s missing
missing = []
for key, p in files.items():
    exists = p.exists()
    print(f"{key}: exists={exists} -> {p}")
    if not exists:
        missing.append((key, p))

if missing:
    print("\n⚠️ Some raw files are missing:")
    for k, p in missing:
        print(f"  - {k}: {p}")
    print("   → Update `files[...]` paths above or put the files in the expected locations.")


PROJECT_ROOT = C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc
Using DATA_INTERIM = C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim
Using DATA_PROCESSED = C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed
k: exists=True -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\raw\UKHLS\k_indresp.sav
l: exists=True -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\raw\UKHLS\l_indresp.sav
n: exists=True -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\raw\UKHLS\n_indresp.sav


In [32]:
# === Section 2: Define per-wave "columns of interest" (ensure 21 with pidp first) ===
import pandas as pd

# Curated requests (note: fixed the stray space before k_hhsize)
k_cols_interest = [
    "pidp",
    "k_hrpid","k_ppid","k_ind5mus_xw","k_indinui_lw",
    "k_intdaty_dv","k_age_dv","k_sex_dv",
    "k_gor_dv","k_urban_dv","k_hhtype_dv","k_hhsize",
    "k_ethn_dv","k_mhealthtyp1",
    "k_jbft_dv","k_jbnssec_dv","k_jbhrs",
    "k_fimnnet_dv","k_sf12pcs_dv","k_scghq1_dv","k_nchild_dv",
]

l_cols_interest = [
    "pidp",
    "l_hrpid","l_ppid","l_ind5mus_xw","l_indinui_lw",
    "l_intdaty_dv","l_age_dv","l_sex_dv",
    "l_gor_dv","l_urban_dv","l_hhtype_dv","l_hhsize",
    "l_ethn_dv","l_mhealthtyp1",
    "l_jbft_dv","l_jbnssec_dv","l_jbhrs",
    "l_fimnnet_dv","l_sf12pcs_dv","l_scghq1_dv","l_nchild_dv",
]

# N uses anxiety = mhgad
n_cols_interest = [
    "pidp",
    "n_hrpid","n_ppid","n_ind5mus_xw","n_indinui_lw",
    "n_intdaty_dv","n_age_dv","n_sex_dv",
    "n_gor_dv","n_urban_dv","n_hhtype_dv","n_hhsize",
    "n_ethn_dv","n_mhgad",
    "n_jbft_dv","n_jbnssec_dv","n_jbhrs",
    "n_fimnnet_dv","n_sf12pcs_dv","n_scghq1_dv","n_nchild_dv",
]

def _read_all_cols(prefix: str) -> list[str]:
    """Load the full SPSS header list saved earlier by your 'show_columns' cell."""
    path = DATA_INTERIM / f"ukhls_{prefix}_all_columns.csv"
    s = pd.read_csv(path)["column"].astype(str).str.strip()
    return s.tolist()

def ensure_21_with_pidp(prefix: str, cols: list[str]) -> list[str]:
    """
    - Strip whitespace, drop duplicates (keep order)
    - Ensure 'pidp' present and first (if available in header)
    - Validate against saved header list for the wave
    - Cap to exactly 21 columns (your target)
    - Persist list to data/interim for later cells
    """
    requested = []
    seen = set()
    for c in [c.strip() for c in cols if isinstance(c, str)]:
        if c and c not in seen:
            requested.append(c); seen.add(c)

    header = set(_read_all_cols(prefix))

    # Ensure pidp first if available
    if "pidp" in header:
        requested = ["pidp"] + [c for c in requested if c != "pidp"]

    # Report any requested columns not in the header (helps catch typos)
    missing_in_header = [c for c in requested if c not in header]
    if missing_in_header:
        print(f"⚠️ {prefix.upper()} missing in header: {missing_in_header}")

    # Keep only those that actually exist in the header
    requested = [c for c in requested if c in header]

    # Cap to 21
    requested = requested[:21]

    # Persist & echo
    out = DATA_INTERIM / f"ukhls_{prefix}_columns_of_interest.csv"
    pd.Series(requested, name="column").to_frame().to_csv(out, index=False)
    print(f"{prefix.upper()} -> {len(requested)} columns")
    print(requested)
    print("-" * 120)
    return requested

k_cols_interest = ensure_21_with_pidp("k", k_cols_interest)
l_cols_interest = ensure_21_with_pidp("l", l_cols_interest)
n_cols_interest = ensure_21_with_pidp("n", n_cols_interest)

K -> 21 columns
['pidp', 'k_hrpid', 'k_ppid', 'k_ind5mus_xw', 'k_indinui_lw', 'k_intdaty_dv', 'k_age_dv', 'k_sex_dv', 'k_gor_dv', 'k_urban_dv', 'k_hhtype_dv', 'k_hhsize', 'k_ethn_dv', 'k_mhealthtyp1', 'k_jbft_dv', 'k_jbnssec_dv', 'k_jbhrs', 'k_fimnnet_dv', 'k_sf12pcs_dv', 'k_scghq1_dv', 'k_nchild_dv']
------------------------------------------------------------------------------------------------------------------------
L -> 21 columns
['pidp', 'l_hrpid', 'l_ppid', 'l_ind5mus_xw', 'l_indinui_lw', 'l_intdaty_dv', 'l_age_dv', 'l_sex_dv', 'l_gor_dv', 'l_urban_dv', 'l_hhtype_dv', 'l_hhsize', 'l_ethn_dv', 'l_mhealthtyp1', 'l_jbft_dv', 'l_jbnssec_dv', 'l_jbhrs', 'l_fimnnet_dv', 'l_sf12pcs_dv', 'l_scghq1_dv', 'l_nchild_dv']
------------------------------------------------------------------------------------------------------------------------
N -> 21 columns
['pidp', 'n_hrpid', 'n_ppid', 'n_ind5mus_xw', 'n_indinui_lw', 'n_intdaty_dv', 'n_age_dv', 'n_sex_dv', 'n_gor_dv', 'n_urban_dv', 'n_hhtyp

In [39]:
# === Save per-wave columns-of-interest to Parquet, save preview, then reload & show 10 rows ===
import pandas as pd
import pyreadstat
from pathlib import Path

OVERWRITE = True  # set False to skip writing if files already exist

IN_DIR  = DATA_INTERIM  # where your *_all_columns.csv live
OUT_DIR = DATA_INTERIM  # saving Parquet & previews here too
OUT_DIR.mkdir(parents=True, exist_ok=True)

keep_maps = {"k": k_cols_interest, "l": l_cols_interest, "n": n_cols_interest}
raw_paths = {"k": files["k"], "l": files["l"], "n": files["n"]}

def _arrow_safe_parquet(df: pd.DataFrame, out_path: Path):
    """
    Normalize dtypes that can bother Arrow and save to Parquet.
    """
    df2 = df.copy()
    for c in df2.columns:
        # convert categoricals/objects to string for Arrow friendliness
        if hasattr(pd.api.types, "CategoricalDtype") and isinstance(df2[c].dtype, pd.CategoricalDtype):
            df2[c] = df2[c].astype("string")
        elif pd.api.types.is_object_dtype(df2[c]):
            df2[c] = df2[c].astype("string")
    df2.to_parquet(out_path, index=False)
    return out_path

def save_wave_subset(prefix: str) -> Path:
    """
    Read *only* the requested columns for a wave, save Parquet + preview CSV.
    Return the Parquet path.
    """
    usecols = keep_maps[prefix]
    df, _ = pyreadstat.read_sav(
        raw_paths[prefix],
        usecols=usecols,
        apply_value_formats=False
    )
    print(f"{prefix.upper()} -> shape {df.shape}")

    pq_path = OUT_DIR / f"ukhls_{prefix}_columns_of_interest.parquet"
    prev_path = OUT_DIR / f"ukhls_{prefix}_preview_head10.csv"

    if pq_path.exists() and not OVERWRITE:
        print(f"{prefix.upper()} exists -> {pq_path} (skip write; set OVERWRITE=True to regenerate)")
    else:
        _arrow_safe_parquet(df, pq_path)
        print(f"Saved Parquet -> {pq_path}")

        df.head(10).to_csv(prev_path, index=False)
        print(f"Saved preview (10 rows) -> {prev_path}")

    return pq_path

print("=== Reading & saving per-wave 'columns of interest' to Parquet (with previews) ===")
k_pq = save_wave_subset("k")
l_pq = save_wave_subset("l")
n_pq = save_wave_subset("n")

# ---- Reload from saved Parquet and display 10 rows for each ----
def show_head_from_parquet(path: Path, title: str, n: int = 10):
    df = pd.read_parquet(path)
    print(f"\n{title} (loaded from Parquet) — head({n}) [{df.shape[0]} rows, {df.shape[1]} cols]")
    display(df.head(n))

show_head_from_parquet(k_pq, "K")
show_head_from_parquet(l_pq, "L")
show_head_from_parquet(n_pq, "N")

=== Reading & saving per-wave 'columns of interest' to Parquet (with previews) ===
K -> shape (32008, 21)
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim\ukhls_k_columns_of_interest.parquet
Saved preview (10 rows) -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim\ukhls_k_preview_head10.csv
K -> shape (32008, 21)
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim\ukhls_k_columns_of_interest.parquet
Saved preview (10 rows) -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim\ukhls_k_preview_head10.csv
L -> shape (29271, 21)
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim\ukhls_l_columns_of_interest.parquet
Saved preview (10 rows) -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\interim\ukhls_l_preview_head10.csv
L -> shape (29271, 21)
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\inter

,pidp,k_mhealthtyp1,k_hhsize,k_jbhrs,k_sex_dv,k_age_dv,k_intdaty_dv,k_ethn_dv,k_fimnnet_dv,k_gor_dv,...,k_hhtype_dv,k_nchild_dv,k_hrpid,k_ppid,k_jbft_dv,k_jbnssec_dv,k_scghq1_dv,k_sf12pcs_dv,k_indinui_lw,k_ind5mus_xw
0,68006127.0,-8.0,2.0,-8.0,2.0,49.0,2019.0,1.0,0.000000,1.0,...,6.0,0.0,68006127.0,68020564.0,-8.0,-8.0,16.0,26.41,1.703009,0.0
1,68020564.0,-8.0,2.0,-8.0,1.0,48.0,2019.0,1.0,1565.670044,1.0,...,6.0,0.0,68006127.0,68006127.0,-8.0,-8.0,12.0,34.68,0.000000,0.0
2,68008847.0,-8.0,1.0,39.0,2.0,61.0,2019.0,1.0,2134.000000,1.0,...,2.0,0.0,68008847.0,-8.0,1.0,14.0,9.0,44.20,0.794699,0.0
3,68009527.0,-8.0,4.0,39.0,1.0,41.0,2019.0,1.0,2043.000000,1.0,...,11.0,2.0,68009527.0,68061288.0,1.0,15.0,16.0,60.48,0.962017,0.0
4,68061288.0,-8.0,4.0,-8.0,2.0,33.0,2019.0,1.0,149.250000,1.0,...,11.0,2.0,68009527.0,68009527.0,-8.0,-8.0,11.0,54.23,0.000000,0.0
5,68010887.0,-8.0,2.0,32.0,2.0,55.0,2019.0,1.0,1250.000000,1.0,...,6.0,0.0,68068082.0,68068082.0,1.0,25.0,9.0,57.28,1.055802,0.0
6,68068082.0,-8.0,2.0,-8.0,1.0,58.0,2019.0,1.0,690.859985,1.0,...,6.0,0.0,68068082.0,68010887.0,1.0,19.0,9.0,56.15,0.000000,0.0
7,68014287.0,-8.0,3.0,-8.0,2.0,49.0,2019.0,1.0,715.869995,1.0,...,18.0,1.0,68014287.0,-8.0,-8.0,-8.0,23.0,44.37,0.000000,0.0
8,68020407.0,-8.0,2.0,-8.0,2.0,82.0,2019.0,1.0,1621.670044,1.0,...,17.0,0.0,68020407.0,-8.0,-8.0,-8.0,13.0,18.07,0.833442,0.0
9,68028575.0,-8.0,4.0,-8.0,2.0,28.0,2019.0,1.0,0.000000,1.0,...,11.0,2.0,68157166.0,68157166.0,-8.0,-8.0,6.0,56.71,1.029255,0.0



L (loaded from Parquet) — head(10) [29271 rows, 21 cols]


,pidp,l_mhealthtyp1,l_hhsize,l_jbhrs,l_sex_dv,l_age_dv,l_intdaty_dv,l_ethn_dv,l_fimnnet_dv,l_gor_dv,...,l_hhtype_dv,l_nchild_dv,l_hrpid,l_ppid,l_jbft_dv,l_jbnssec_dv,l_scghq1_dv,l_sf12pcs_dv,l_indinui_lw,l_ind5mus_xw
0,68008847.0,-8.0,1.0,39.0,2.0,62.0,2020.0,1.0,2288.000000,1.0,...,2.0,0.0,68008847.0,-8.0,1.0,14.0,12.0,37.58,0.739967,0.0
1,68009527.0,-8.0,4.0,36.5,1.0,43.0,2020.0,1.0,2060.000000,1.0,...,11.0,2.0,68034180.0,68061288.0,1.0,15.0,11.0,56.37,0.950116,0.0
2,68061288.0,-8.0,4.0,10.0,2.0,34.0,2020.0,1.0,474.000000,1.0,...,11.0,2.0,68034180.0,68009527.0,2.0,24.0,15.0,61.73,0.000000,0.0
3,68010887.0,-8.0,2.0,32.0,2.0,56.0,2020.0,1.0,1200.000000,1.0,...,6.0,0.0,68010887.0,68068082.0,1.0,25.0,11.0,51.64,0.998976,0.0
4,68068082.0,-8.0,2.0,-8.0,1.0,59.0,2020.0,1.0,3275.909912,1.0,...,6.0,0.0,68010887.0,68010887.0,1.0,17.0,10.0,56.15,0.000000,0.0
5,68028575.0,-8.0,4.0,-8.0,2.0,28.0,2020.0,1.0,0.000000,1.0,...,11.0,2.0,68095380.0,68157166.0,-8.0,-8.0,8.0,57.76,1.031810,0.0
6,68157166.0,-8.0,4.0,38.0,1.0,35.0,2020.0,1.0,2974.000000,1.0,...,11.0,2.0,68095380.0,68028575.0,1.0,8.0,7.0,57.47,0.000000,0.0
7,68029927.0,-8.0,5.0,-8.0,2.0,48.0,2020.0,1.0,89.699997,1.0,...,20.0,0.0,68029939.0,68029931.0,-8.0,-8.0,9.0,53.79,0.000000,0.0
8,68029939.0,-8.0,5.0,-8.0,1.0,16.0,2020.0,1.0,0.000000,1.0,...,20.0,0.0,68029939.0,-8.0,-8.0,-8.0,8.0,57.76,0.000000,0.0
9,68149808.0,-8.0,5.0,30.0,2.0,23.0,2020.0,1.0,865.000000,1.0,...,20.0,0.0,68029939.0,-8.0,1.0,7.0,8.0,56.15,0.000000,0.0



N (loaded from Parquet) — head(10) [35471 rows, 21 cols]


,pidp,n_mhgad,n_hhsize,n_jbhrs,n_sex_dv,n_age_dv,n_intdaty_dv,n_ethn_dv,n_fimnnet_dv,n_gor_dv,...,n_hhtype_dv,n_nchild_dv,n_hrpid,n_ppid,n_jbft_dv,n_scghq1_dv,n_sf12pcs_dv,n_indinui_lw,n_ind5mus_xw,n_jbnssec_dv
0,22445.0,2.0,4.0,28.0,2.0,37.0,2022.0,1.0,1857.079956,8.0,...,11.0,2.0,276841780.0,277059298.0,1.0,24.0,62.83,0.0,0.0,2.0
1,29925.0,2.0,3.0,29.0,2.0,45.0,2022.0,1.0,2378.750000,7.0,...,5.0,2.0,622866606.0,-8.0,2.0,23.0,65.47,0.0,0.0,14.0
2,76165.0,2.0,4.0,35.0,2.0,39.0,2022.0,1.0,3206.000000,5.0,...,11.0,2.0,141045780.0,142378492.0,1.0,12.0,57.20,0.0,0.0,11.0
3,280165.0,2.0,4.0,-8.0,2.0,43.0,2022.0,1.0,94.470001,8.0,...,20.0,1.0,783876922.0,756200970.0,-8.0,16.0,58.55,0.0,0.0,-8.0
4,469205.0,2.0,3.0,16.0,2.0,32.0,2022.0,1.0,2056.080078,4.0,...,5.0,2.0,414412580.0,-8.0,2.0,16.0,49.93,0.0,0.0,25.0
5,599765.0,2.0,3.0,37.0,2.0,35.0,2022.0,1.0,2839.649902,5.0,...,10.0,1.0,209943344.0,210167702.0,1.0,6.0,56.15,0.0,0.0,2.0
6,732365.0,1.0,3.0,-8.0,1.0,37.0,2022.0,1.0,838.500000,2.0,...,19.0,0.0,732365.0,-8.0,-8.0,35.0,45.49,0.0,0.0,-8.0
7,1587125.0,2.0,1.0,37.0,2.0,56.0,2022.0,1.0,2290.000000,1.0,...,3.0,0.0,1587125.0,-8.0,1.0,15.0,37.37,0.0,0.0,7.0
8,2888645.0,2.0,1.0,38.0,2.0,33.0,2022.0,1.0,2300.000000,11.0,...,3.0,0.0,2888645.0,-8.0,1.0,6.0,56.15,0.0,0.0,2.0
9,3424485.0,2.0,5.0,-8.0,2.0,86.0,2022.0,1.0,1092.750000,6.0,...,20.0,0.0,103550562.0,-8.0,-8.0,10.0,33.23,0.0,0.0,-8.0


In [28]:
# === Section 3B: Preview ALL SPSS columns for K -> L -> N (sequential) ===
import pandas as pd
import pyreadstat
from pathlib import Path

def list_spss_columns(path: Path) -> list[str]:
    """Read header only and return full column list (no truncation)."""
    _, meta = pyreadstat.read_sav(path, row_limit=1)
    return list(meta.column_names)

def show_columns(prefix: str, path: Path):
    cols = list_spss_columns(path)
    print(f"{prefix.upper()} Columns: {len(cols)} total")
    print(cols)
    # Persist for later auditing/joins with Section 2
    DATA_INTERIM.mkdir(parents=True, exist_ok=True)
    pd.Series(cols, name="column").to_frame().to_csv(
        DATA_INTERIM / f"ukhls_{prefix}_all_columns.csv", index=False
    )
    print("-" * 120)

print("=== SPSS full headers (one pass each) ===")
show_columns("k", files["k"])
show_columns("l", files["l"])
show_columns("n", files["n"])


=== SPSS full headers (one pass each) ===
K Columns: 3395 total
['pidp', 'pid', 'k_hidp', 'k_pno', 'k_hhorig', 'k_memorig', 'k_psu', 'k_strata', 'k_sampst', 'k_month', 'k_quarter', 'k_ivfio', 'k_ioutcome', 'k_sex', 'k_dvage', 'k_birthy', 'k_chkwebdobd', 'k_chkwebdoby', 'k_chkrespweb1', 'k_chkrespweb2', 'k_chkrespweb3', 'k_chkrespweb4', 'k_chkwebsex', 'k_nchunder16', 'k_nch5to15', 'k_nch10to15', 'k_nch10', 'k_nunder16abs', 'k_n1619abs', 'k_nchresp', 'k_nch14resp', 'k_nch415resp', 'k_nch3resp', 'k_nch5resp', 'k_nch8resp', 'k_respchild1', 'k_respchild2', 'k_respchild3', 'k_respchild4', 'k_respchild5', 'k_respchild6', 'k_respchild7', 'k_respchild8', 'k_respchild9', 'k_respchild10', 'k_respchild11', 'k_respchild12', 'k_respchild13', 'k_respchild14', 'k_respchild15', 'k_respchild16', 'k_nadoptch', 'k_adoptchonly', 'k_nnatch', 'k_nstepch', 'k_sibling', 'k_pwcu18abs1', 'k_pwcu18abs2', 'k_pwcu18abs3', 'k_pwcu18abs4', 'k_pwcu18abs5', 'k_pwcu18abs6', 'k_pwcu18abs7', 'k_pwcu18abs8', 'k_pwcu18abs9'

In [27]:
# === Section 3B: Preview ALL SPSS columns for K -> L -> N (sequential) ===
import pandas as pd
import pyreadstat
from pathlib import Path

def list_spss_columns(path: Path) -> list[str]:
    """Read header only and return full column list (no truncation)."""
    _, meta = pyreadstat.read_sav(path, row_limit=1)
    return list(meta.column_names)

def show_columns(prefix: str, path: Path):
    cols = list_spss_columns(path)
    print(f"{prefix.upper()} Columns: {len(cols)} total")
    print(cols)
    # Persist for later auditing/joins with Section 2
    DATA_INTERIM.mkdir(parents=True, exist_ok=True)
    pd.Series(cols, name="column").to_frame().to_csv(
        DATA_INTERIM / f"ukhls_{prefix}_all_columns.csv", index=False
    )
    print("-" * 120)

print("=== SPSS full headers (one pass each) ===")
show_columns("k", files["k"])
show_columns("l", files["l"])
show_columns("n", files["n"])

=== SPSS full headers (one pass each) ===
K Columns: 3395 total
['pidp', 'pid', 'k_hidp', 'k_pno', 'k_hhorig', 'k_memorig', 'k_psu', 'k_strata', 'k_sampst', 'k_month', 'k_quarter', 'k_ivfio', 'k_ioutcome', 'k_sex', 'k_dvage', 'k_birthy', 'k_chkwebdobd', 'k_chkwebdoby', 'k_chkrespweb1', 'k_chkrespweb2', 'k_chkrespweb3', 'k_chkrespweb4', 'k_chkwebsex', 'k_nchunder16', 'k_nch5to15', 'k_nch10to15', 'k_nch10', 'k_nunder16abs', 'k_n1619abs', 'k_nchresp', 'k_nch14resp', 'k_nch415resp', 'k_nch3resp', 'k_nch5resp', 'k_nch8resp', 'k_respchild1', 'k_respchild2', 'k_respchild3', 'k_respchild4', 'k_respchild5', 'k_respchild6', 'k_respchild7', 'k_respchild8', 'k_respchild9', 'k_respchild10', 'k_respchild11', 'k_respchild12', 'k_respchild13', 'k_respchild14', 'k_respchild15', 'k_respchild16', 'k_nadoptch', 'k_adoptchonly', 'k_nnatch', 'k_nstepch', 'k_sibling', 'k_pwcu18abs1', 'k_pwcu18abs2', 'k_pwcu18abs3', 'k_pwcu18abs4', 'k_pwcu18abs5', 'k_pwcu18abs6', 'k_pwcu18abs7', 'k_pwcu18abs8', 'k_pwcu18abs9'

In [15]:
# === Cell 2c (FROM-SAVED ONLY): validate & fix 21-column lists, then preview ===
# Consumes the files written by your previous "save per-wave columns of interest" cell.
import pandas as pd
from pathlib import Path

# Where the previous cell wrote the files:
SAVED_DIR = PROJECT_ROOT / "data" / "interim"   # keep in sync with the cell above
SAVED_DIR.mkdir(parents=True, exist_ok=True)

# Where to write the audited (final) 21-col lists for downstream cells:
DATA_INTERIM = PROJECT_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

# --- small, robust loader that prefers Parquet but falls back to CSV if needed ---
def _load_saved(prefix: str) -> pd.DataFrame:
    pqt = SAVED_DIR / f"ukhls_{prefix}_columns_of_interest.parquet"
    csv = SAVED_DIR / f"ukhls_{prefix}_columns_of_interest.csv"
    if pqt.exists():
        try:
            return pd.read_parquet(pqt)
        except Exception as e:
            print(f"[{prefix.upper()}] Parquet read failed ({type(e).__name__}: {e}). Falling back to CSV if present.")
    if csv.exists():
        return pd.read_csv(csv)
    raise FileNotFoundError(
        f"No saved file for wave '{prefix}' in {SAVED_DIR} "
        f"(expected one of: {pqt.name} or {csv.name})."
    )

# --- ensure 21 columns with pidp, but only using what exists in the SAVED files ---
def ensure_21_with_pidp_from_saved(prefix: str, curated_cols: list[str]) -> list[str]:
    df = _load_saved(prefix)
    saved_cols = set(map(str, df.columns))

    # Start from curated intent (copy to avoid side-effects)
    cols = curated_cols[:]

    # Put pidp first if present in saved data
    if "pidp" in saved_cols:
        cols = [c for c in cols if c != "pidp"]
        cols = ["pidp"] + cols

    # Keep only columns that actually exist in the saved file
    present = [c for c in cols if c in saved_cols]

    # Cap to 21 (your contract)
    final_cols = present[:21]

    # Persist the final audited list for downstream cells
    out = DATA_INTERIM / f"ukhls_{prefix}_columns_of_interest.csv"
    pd.DataFrame({"column": final_cols}).to_csv(out, index=False)

    # Report
    missing = [c for c in cols if c not in saved_cols]
    print(f"{prefix.upper()} -> saved shape {df.shape}; selected {len(final_cols)} columns")
    if missing:
        print(f"  dropped (not in saved): {missing}")
    print(f"  final 21 (or fewer if unavailable): {final_cols}\n")
    return final_cols

# ---- Use your curated lists from the cell above (these variables already exist): ----
# k_cols_interest, l_cols_interest, n_cols_interest

k_cols_interest = ensure_21_with_pidp_from_saved("k", k_cols_interest)
l_cols_interest = ensure_21_with_pidp_from_saved("l", l_cols_interest)
n_cols_interest = ensure_21_with_pidp_from_saved("n", n_cols_interest)

# ---- Quick sanity previews (10 rows each) from SAVED files only ----
k_sel = _load_saved("k")
l_sel = _load_saved("l")
n_sel = _load_saved("n")

print("K — preview (10 rows):")
display(k_sel[k_cols_interest].head(15))
print("L — preview (10 rows):")
display(l_sel[l_cols_interest].head(15))
print("N — preview (10 rows):")
display(n_sel[n_cols_interest].head(15))


K -> saved shape (32008, 21); selected 21 columns
  final 21 (or fewer if unavailable): ['pidp', 'k_hrpid', 'k_ppid', 'k_ind5mus_xw', 'k_indinui_lw', 'k_intdaty_dv', 'k_age_dv', 'k_sex_dv', 'k_gor_dv', 'k_urban_dv', 'k_hhtype_dv', 'k_hhsize', 'k_ethn_dv', 'k_mhealthtyp1', 'k_jbft_dv', 'k_jbnssec_dv', 'k_jbhrs', 'k_fimnnet_dv', 'k_sf12pcs_dv', 'k_scghq1_dv', 'k_nchild_dv']

L -> saved shape (29271, 21); selected 21 columns
  final 21 (or fewer if unavailable): ['pidp', 'l_hrpid', 'l_ppid', 'l_ind5mus_xw', 'l_indinui_lw', 'l_intdaty_dv', 'l_age_dv', 'l_sex_dv', 'l_gor_dv', 'l_urban_dv', 'l_hhtype_dv', 'l_hhsize', 'l_ethn_dv', 'l_mhealthtyp1', 'l_jbft_dv', 'l_jbnssec_dv', 'l_jbhrs', 'l_fimnnet_dv', 'l_sf12pcs_dv', 'l_scghq1_dv', 'l_nchild_dv']

N -> saved shape (35471, 21); selected 21 columns
  final 21 (or fewer if unavailable): ['pidp', 'n_hrpid', 'n_ppid', 'n_ind5mus_xw', 'n_indinui_lw', 'n_intdaty_dv', 'n_age_dv', 'n_sex_dv', 'n_gor_dv', 'n_urban_dv', 'n_hhtype_dv', 'n_hhsize', 'n_et

,pidp,k_hrpid,k_ppid,k_ind5mus_xw,k_indinui_lw,k_intdaty_dv,k_age_dv,k_sex_dv,k_gor_dv,k_urban_dv,...,k_hhsize,k_ethn_dv,k_mhealthtyp1,k_jbft_dv,k_jbnssec_dv,k_jbhrs,k_fimnnet_dv,k_sf12pcs_dv,k_scghq1_dv,k_nchild_dv
0,68006127.0,68006127.0,68020564.0,0.0,1.703009,2019.0,49.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,26.41,16.0,0.0
1,68020564.0,68006127.0,68006127.0,0.0,0.000000,2019.0,48.0,1.0,1.0,1.0,...,2.0,1.0,-8.0,-8.0,-8.0,-8.0,1565.670044,34.68,12.0,0.0
2,68008847.0,68008847.0,-8.0,0.0,0.794699,2019.0,61.0,2.0,1.0,1.0,...,1.0,1.0,-8.0,1.0,14.0,39.0,2134.000000,44.20,9.0,0.0
3,68009527.0,68009527.0,68061288.0,0.0,0.962017,2019.0,41.0,1.0,1.0,1.0,...,4.0,1.0,-8.0,1.0,15.0,39.0,2043.000000,60.48,16.0,2.0
4,68061288.0,68009527.0,68009527.0,0.0,0.000000,2019.0,33.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,-8.0,-8.0,-8.0,149.250000,54.23,11.0,2.0
5,68010887.0,68068082.0,68068082.0,0.0,1.055802,2019.0,55.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,25.0,32.0,1250.000000,57.28,9.0,0.0
6,68068082.0,68068082.0,68010887.0,0.0,0.000000,2019.0,58.0,1.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,19.0,-8.0,690.859985,56.15,9.0,0.0
7,68014287.0,68014287.0,-8.0,0.0,0.000000,2019.0,49.0,2.0,1.0,1.0,...,3.0,1.0,-8.0,-8.0,-8.0,-8.0,715.869995,44.37,23.0,1.0
8,68020407.0,68020407.0,-8.0,0.0,0.833442,2019.0,82.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,-8.0,-8.0,-8.0,1621.670044,18.07,13.0,0.0
9,68028575.0,68157166.0,68157166.0,0.0,1.029255,2019.0,28.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,56.71,6.0,2.0


L — preview (10 rows):


,pidp,l_hrpid,l_ppid,l_ind5mus_xw,l_indinui_lw,l_intdaty_dv,l_age_dv,l_sex_dv,l_gor_dv,l_urban_dv,...,l_hhsize,l_ethn_dv,l_mhealthtyp1,l_jbft_dv,l_jbnssec_dv,l_jbhrs,l_fimnnet_dv,l_sf12pcs_dv,l_scghq1_dv,l_nchild_dv
0,68008847.0,68008847.0,-8.0,0.0,0.739967,2020.0,62.0,2.0,1.0,1.0,...,1.0,1.0,-8.0,1.0,14.0,39.0,2288.000000,37.58,12.0,0.0
1,68009527.0,68034180.0,68061288.0,0.0,0.950116,2020.0,43.0,1.0,1.0,1.0,...,4.0,1.0,-8.0,1.0,15.0,36.5,2060.000000,56.37,11.0,2.0
2,68061288.0,68034180.0,68009527.0,0.0,0.000000,2020.0,34.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,2.0,24.0,10.0,474.000000,61.73,15.0,2.0
3,68010887.0,68010887.0,68068082.0,0.0,0.998976,2020.0,56.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,25.0,32.0,1200.000000,51.64,11.0,0.0
4,68068082.0,68010887.0,68010887.0,0.0,0.000000,2020.0,59.0,1.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,17.0,-8.0,3275.909912,56.15,10.0,0.0
5,68028575.0,68095380.0,68157166.0,0.0,1.031810,2020.0,28.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,2.0
6,68157166.0,68095380.0,68028575.0,0.0,0.000000,2020.0,35.0,1.0,1.0,1.0,...,4.0,1.0,-8.0,1.0,8.0,38.0,2974.000000,57.47,7.0,2.0
7,68029927.0,68029939.0,68029931.0,0.0,0.000000,2020.0,48.0,2.0,1.0,1.0,...,5.0,1.0,-8.0,-8.0,-8.0,-8.0,89.699997,53.79,9.0,0.0
8,68029939.0,68029939.0,-8.0,0.0,0.000000,2020.0,16.0,1.0,1.0,1.0,...,5.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,0.0
9,68149808.0,68029939.0,-8.0,0.0,0.000000,2020.0,23.0,2.0,1.0,1.0,...,5.0,1.0,-8.0,1.0,7.0,30.0,865.000000,56.15,8.0,0.0


N — preview (10 rows):


,pidp,n_hrpid,n_ppid,n_ind5mus_xw,n_indinui_lw,n_intdaty_dv,n_age_dv,n_sex_dv,n_gor_dv,n_urban_dv,...,n_hhsize,n_ethn_dv,n_mhgad,n_jbft_dv,n_jbnssec_dv,n_jbhrs,n_fimnnet_dv,n_sf12pcs_dv,n_scghq1_dv,n_nchild_dv
0,22445.0,276841780.0,277059298.0,0.0,0.0,2022.0,37.0,2.0,8.0,1.0,...,4.0,1.0,2.0,1.0,2.0,28.0,1857.079956,62.83,24.0,2.0
1,29925.0,622866606.0,-8.0,0.0,0.0,2022.0,45.0,2.0,7.0,1.0,...,3.0,1.0,2.0,2.0,14.0,29.0,2378.750000,65.47,23.0,2.0
2,76165.0,141045780.0,142378492.0,0.0,0.0,2022.0,39.0,2.0,5.0,1.0,...,4.0,1.0,2.0,1.0,11.0,35.0,3206.000000,57.20,12.0,2.0
3,280165.0,783876922.0,756200970.0,0.0,0.0,2022.0,43.0,2.0,8.0,2.0,...,4.0,1.0,2.0,-8.0,-8.0,-8.0,94.470001,58.55,16.0,1.0
4,469205.0,414412580.0,-8.0,0.0,0.0,2022.0,32.0,2.0,4.0,1.0,...,3.0,1.0,2.0,2.0,25.0,16.0,2056.080078,49.93,16.0,2.0
5,599765.0,209943344.0,210167702.0,0.0,0.0,2022.0,35.0,2.0,5.0,1.0,...,3.0,1.0,2.0,1.0,2.0,37.0,2839.649902,56.15,6.0,1.0
6,732365.0,732365.0,-8.0,0.0,0.0,2022.0,37.0,1.0,2.0,1.0,...,3.0,1.0,1.0,-8.0,-8.0,-8.0,838.500000,45.49,35.0,0.0
7,1587125.0,1587125.0,-8.0,0.0,0.0,2022.0,56.0,2.0,1.0,1.0,...,1.0,1.0,2.0,1.0,7.0,37.0,2290.000000,37.37,15.0,0.0
8,2888645.0,2888645.0,-8.0,0.0,0.0,2022.0,33.0,2.0,11.0,1.0,...,1.0,1.0,2.0,1.0,2.0,38.0,2300.000000,56.15,6.0,0.0
9,3424485.0,103550562.0,-8.0,0.0,0.0,2022.0,86.0,2.0,6.0,2.0,...,5.0,1.0,2.0,-8.0,-8.0,-8.0,1092.750000,33.23,10.0,0.0


In [ ]:
# Preview the first 10 rows of each model-ready DataFrame
print("K model-ready (first 10 rows):")
display(k_mr.head(10))
print("L model-ready (first 10 rows):")
display(l_mr.head(10))
print("N model-ready (first 10 rows):")
display(n_mr.head(10))

In [27]:
# === Cell 1: Setup, load saved columns-of-interest, standardize, save ===
import pandas as pd
from pathlib import Path

# ---- Missing-code map (used later too; keep here for reuse) ----
MISS_LABELS = {
    -9: "missing",
    -8: "inapplicable",
    -7: "proxy/partial",
    -2: "refusal",
    -1: "don't know",
}
MISS_CODES = set(MISS_LABELS.keys())

# ---- Config ----
PROJECT_ROOT = Path(PROJECT_ROOT)        # reuse your session var
IN_DIR  = PROJECT_ROOT / "data" / "interim"     # has ukhls_{k|l|n}_columns_of_interest.(parquet|csv)
OUT_DIR = PROJECT_ROOT / "data" / "processed"   # will write standardized/model-ready/analysis-ready
OUT_DIR.mkdir(parents=True, exist_ok=True)

SAVE_FMT = "parquet"   # "parquet" (preferred) or "csv"
WAVE_NUM = {"k": 11, "l": 12, "n": 14}

# 21 standardized variables (target schema)
STD_COLS = [
    "pidp",
    "hrpid","ppid","ind5mus_xw","indinui_lw",
    "intdaty_dv","age_dv","sex_dv",
    "gor_dv","urban_dv","hhtype_dv","hhsize",
    "ethn_dv","anxiety_raw",
    "jbft_dv","jbnssec_dv","jbhrs",
    "fimnnet_dv","sf12pcs_dv","scghq1_dv","nchild_dv",
]

# ---------- I/O helpers ----------
def _find_input_file(prefix: str) -> Path:
    base = IN_DIR / f"ukhls_{prefix}_columns_of_interest"
    pq, cs = base.with_suffix(".parquet"), base.with_suffix(".csv")
    if pq.exists(): return pq
    if cs.exists(): return cs
    raise FileNotFoundError(f"No input for {prefix}: {pq.name} / {cs.name}")

def _load_df(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".parquet": return pd.read_parquet(path)
    if path.suffix.lower() == ".csv":     return pd.read_csv(path)
    raise ValueError(f"Unsupported file type: {path}")

def _safe_to_parquet_or_csv(df: pd.DataFrame, path_base: Path, fmt: str = "parquet") -> Path:
    """Try Parquet, fall back to CSV. Normalize object/categorical -> string for Arrow friendliness."""
    df2 = df.copy()
    for c in df2.columns:
        if isinstance(df2[c].dtype, pd.CategoricalDtype):
            df2[c] = df2[c].astype("string")
        elif pd.api.types.is_object_dtype(df2[c]):
            df2[c] = df2[c].astype("string")

    if fmt.lower() == "parquet":
        try:
            out = path_base.with_suffix(".parquet")
            df2.to_parquet(out, index=False)
            print(f"Saved Parquet -> {out}")
            return out
        except Exception as e:
            print(f"Parquet failed ({type(e).__name__}: {e}). Falling back to CSV.")
            out = path_base.with_suffix(".csv")
            df.to_csv(out, index=False)
            print(f"Saved CSV -> {out}")
            return out
    elif fmt.lower() == "csv":
        out = path_base.with_suffix(".csv")
        df.to_csv(out, index=False)
        print(f"Saved CSV -> {out}")
        return out
    else:
        raise ValueError("SAVE_FMT must be 'parquet' or 'csv'")

# ---------- Standardize ----------
def standardize_wave(df_in: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """
    Strip wave prefix, build anxiety_raw (mhgad or mhealthtyp1), add wave + wave_num,
    ensure all STD_COLS exist, return only STD_COLS + ['wave','wave_num'].
    Also removes any duplicate columns created by collisions after renaming.
    """
    df = df_in.copy()
    before = df.shape

    # 1) strip prefix
    pref = f"{prefix}_"
    ren = {c: c[len(pref):] for c in df.columns if c.startswith(pref)}
    df = df.rename(columns=ren)

    # 2) drop duplicate columns created by collisions (keep the last)
    if df.columns.duplicated().any():
        dup_names = df.columns[df.columns.duplicated()].tolist()
        print(f"{prefix.upper()} -> duplicate cols after renaming; dropping older copies: {sorted(set(dup_names))}")
        df = df.loc[:, ~df.columns.duplicated(keep="last")]

    # 3) harmonize anxiety_raw only if missing
    if "anxiety_raw" not in df.columns:
        if "mhgad" in df.columns:
            df["anxiety_raw"] = df["mhgad"]
        elif "mhealthtyp1" in df.columns:
            df["anxiety_raw"] = df["mhealthtyp1"]

    # 4) add wave indicators
    df["wave"] = prefix.upper()
    df["wave_num"] = WAVE_NUM[prefix]

    # 5) ensure all STD_COLS exist
    for c in STD_COLS:
        if c not in df.columns:
            df[c] = pd.NA

    # 6) final select + guard duplicates
    final_cols = pd.Index(STD_COLS + ["wave","wave_num"]).drop_duplicates().tolist()
    df = df.loc[:, final_cols]

    after = df.shape
    print(f"{prefix.UPPER()} -> BEFORE: {before} | AFTER (standardized): {after}")
    return df

# ---------- Run (Cell 1) ----------
print("=== Load saved columns-of-interest ===")
k_in = _load_df(_find_input_file("k"))
l_in = _load_df(_find_input_file("l"))
n_in = _load_df(_find_input_file("n"))

print("\n=== Standardize per wave ===")
k_std = standardize_wave(k_in, "k")
l_std = standardize_wave(l_in, "l")
n_std = standardize_wave(n_in, "n")

# Preview
print("\nK std (10 rows):"); display(k_std.head(10))
print("\nL std (10 rows):"); display(l_std.head(10))
print("\nN std (10 rows):"); display(n_std.head(10))

# Save standardized (raw) per-wave
print("\n=== Save standardized frames ===")
k_std_path = _safe_to_parquet_or_csv(k_std, OUT_DIR / "ukhls_k_standardized", SAVE_FMT)
l_std_path = _safe_to_parquet_or_csv(l_std, OUT_DIR / "ukhls_l_standardized", SAVE_FMT)
n_std_path = _safe_to_parquet_or_csv(n_std, OUT_DIR / "ukhls_n_standardized", SAVE_FMT)

print("\nStandardized saved ->", k_std_path, l_std_path, n_std_path)


=== Load saved columns-of-interest ===

=== Standardize per wave ===


AttributeError: 'str' object has no attribute 'UPPER'

In [ ]:
# === Load saved (CSV/Parquet), standardize across waves, dynamically clean, and save both versions ===
import pandas as pd
from pathlib import Path

# ---- Missing-code map ----
MISS_LABELS = {
    -9: "missing",
    -8: "inapplicable",
    -7: "proxy/partial",
    -2: "refusal",
    -1: "don't know",
}
MISS_CODES = set(MISS_LABELS.keys())

# ---- Config ----
PROJECT_ROOT = Path(PROJECT_ROOT)  # reuse your existing var
IN_DIR  = PROJECT_ROOT / "data" / "interim"    # where 'ukhls_{k|l|n}_columns_of_interest.(parquet|csv)' live
OUT_DIR = PROJECT_ROOT / "data" / "processed"  # where standardized & model-ready will be written
OUT_DIR.mkdir(parents=True, exist_ok=True)

SAVE_FMT = "parquet"  # "parquet" (preferred) or "csv"
WAVE_NUM = {"k": 11, "l": 12, "n": 14}

# 21 standardized variables
STD_COLS = [
    "pidp",
    "hrpid","ppid","ind5mus_xw","indinui_lw",
    "intdaty_dv","age_dv","sex_dv",
    "gor_dv","urban_dv","hhtype_dv","hhsize",
    "ethn_dv","anxiety_raw",
    "jbft_dv","jbnssec_dv","jbhrs",
    "fimnnet_dv","sf12pcs_dv","scghq1_dv","nchild_dv",
]

# ---------- I/O helpers ----------
def _find_input_file(prefix: str) -> Path:
    base = IN_DIR / f"ukhls_{prefix}_columns_of_interest"
    pq, cs = base.with_suffix(".parquet"), base.with_suffix(".csv")
    if pq.exists(): return pq
    if cs.exists(): return cs
    raise FileNotFoundError(f"No input for {prefix}: {pq.name} / {cs.name}")

def _load_df(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".parquet":
        return pd.read_parquet(path)
    elif path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    else:
        raise ValueError(f"Unsupported file type: {path}")

def _safe_to_parquet_or_csv(df: pd.DataFrame, path_base: Path, fmt: str = "parquet") -> Path:
    """
    Try Parquet, fall back to CSV. Normalizes object/categorical to string for Arrow friendliness.
    Returns final written path.
    """
    df2 = df.copy()
    for c in df2.columns:
        # avoid deprecated is_categorical_dtype
        if isinstance(df2[c].dtype, pd.CategoricalDtype):
            df2[c] = df2[c].astype("string")
        elif pd.api.types.is_object_dtype(df2[c]):
            df2[c] = df2[c].astype("string")

    if fmt.lower() == "parquet":
        try:
            out = path_base.with_suffix(".parquet")
            df2.to_parquet(out, index=False)
            print(f"Saved Parquet -> {out}")
            return out
        except Exception as e:
            print(f"Parquet failed ({type(e).__name__}: {e}). Falling back to CSV.")
            out = path_base.with_suffix(".csv")
            df.to_csv(out, index=False)
            print(f"Saved CSV -> {out}")
            return out
    elif fmt.lower() == "csv":
        out = path_base.with_suffix(".csv")
        df.to_csv(out, index=False)
        print(f"Saved CSV -> {out}")
        return out
    else:
        raise ValueError("SAVE_FMT must be 'parquet' or 'csv'")

# ---------- Standardize ----------
def standardize_wave(df_in: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """
    Strip wave prefix, build anxiety_raw (mhgad or mhealthtyp1), add wave + wave_num,
    ensure all STD_COLS exist, return only STD_COLS + ['wave','wave_num'].
    Also removes any duplicate columns created by collisions after renaming.
    """
    df = df_in.copy()
    before = df.shape

    # 1) strip prefix
    pref = f"{prefix}_"
    ren = {c: c[len(pref):] for c in df.columns if c.startswith(pref)}
    df = df.rename(columns=ren)

    # 2) DROP duplicate columns created by collisions (keep the last)
    if df.columns.duplicated().any():
        dup_names = df.columns[df.columns.duplicated()].tolist()
        print(f"{prefix.upper()} -> found duplicate cols after renaming; dropping older copies: {sorted(set(dup_names))}")
        df = df.loc[:, ~df.columns.duplicated(keep="last")]

    # 3) harmonize anxiety_raw only if missing
    if "anxiety_raw" not in df.columns:
        if "mhgad" in df.columns:
            df["anxiety_raw"] = df["mhgad"]
        elif "mhealthtyp1" in df.columns:
            df["anxiety_raw"] = df["mhealthtyp1"]

    # 4) add wave indicators
    df["wave"] = prefix.upper()
    df["wave_num"] = WAVE_NUM[prefix]

    # 5) ensure all STD_COLS exist
    for c in STD_COLS:
        if c not in df.columns:
            df[c] = pd.NA

    # 6) final select (unique, canonical ordering)
    final_cols = STD_COLS + ["wave", "wave_num"]
    # guard again in case user files carried extra duplicates
    final_cols = pd.Index(final_cols).drop_duplicates().tolist()
    df = df.loc[:, final_cols]

    after = df.shape
    print(f"{prefix.upper()} -> BEFORE: {before} | AFTER (standardized): {after}")
    return df


# ---------- Dynamic cleaner ----------
def add_clean_and_flags_dynamic(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    """
    For each numeric column that actually contains a special negative, add:
      <col>_clean     (special negatives -> NA)
      <col>_missflag  ('missing'/'inapplicable'/.../'observed')
    """
    out = df.copy()
    cleaned = []

    # consider numeric-ish columns (works if they are float/int or parseable)
    num_cols = []
    for c in out.columns:
        # try to_numeric on a small sample to avoid full copy; fallback to dtype check
        try:
            _ = pd.to_numeric(out[c], errors="coerce")
            num_cols.append(c)
        except Exception:
            if pd.api.types.is_numeric_dtype(out[c]):
                num_cols.append(c)

    for c in num_cols:
        v = pd.to_numeric(out[c], errors="coerce")
        if v.isin(MISS_CODES).any():
            out[f"{c}_clean"] = v.where(~v.isin(MISS_CODES), pd.NA)
            # flags with human labels (keep 'observed' for non-miss codes)
            flag = v.map(lambda x: ("NaN" if pd.isna(x) else MISS_LABELS.get(int(x), "observed")))
            out[f"{c}_missflag"] = pd.Categorical(flag)
            cleaned.append(c)

    return out, cleaned

# ---------- Run ----------
print("=== Load saved columns-of-interest ===")
k_in = _load_df(_find_input_file("k"))
l_in = _load_df(_find_input_file("l"))
n_in = _load_df(_find_input_file("n"))

print("\n=== Standardize per wave ===")
k_std = standardize_wave(k_in, "k")
l_std = standardize_wave(l_in, "l")
n_std = standardize_wave(n_in, "n")

# Quick previews
print("\nK std (10 rows):"); display(k_std.head(10))
print("\nL std (10 rows):"); display(l_std.head(10))
print("\nN std (10 rows):"); display(n_std.head(10))

# Save standardized (raw) per-wave
print("\n=== Save standardized frames ===")
k_std_path = _safe_to_parquet_or_csv(k_std, OUT_DIR / "ukhls_k_standardized", SAVE_FMT)
l_std_path = _safe_to_parquet_or_csv(l_std, OUT_DIR / "ukhls_l_standardized", SAVE_FMT)
n_std_path = _safe_to_parquet_or_csv(n_std, OUT_DIR / "ukhls_n_standardized", SAVE_FMT)

# Build model-ready (dynamic clean)
print("\n=== Dynamic clean (add *_clean and *_missflag only where needed) ===")
k_mr, k_cleaned = add_clean_and_flags_dynamic(k_std)
l_mr, l_cleaned = add_clean_and_flags_dynamic(l_std)
n_mr, n_cleaned = add_clean_and_flags_dynamic(n_std)

print(f"Shapes AFTER dynamic clean -> K: {k_mr.shape}  L: {l_mr.shape}  N: {n_mr.shape}")
print("K cleaned columns:", k_cleaned)
print("L cleaned columns:", l_cleaned)
print("N cleaned columns:", n_cleaned)

# Optional: previews of only the cleaned columns
def _preview_clean(df, cleaned_cols, title, n=10):
    cols = ["pidp","wave","wave_num","intdaty_dv"]
    for c in cleaned_cols:
        cols += [c, f"{c}_clean", f"{c}_missflag"]
    cols = [c for c in cols if c in df.columns]
    print(f"\n{title} — first {n} rows (only cleaned columns)")
    display(df[cols].head(n))

_preview_clean(k_mr, k_cleaned, "K (model-ready)")
_preview_clean(l_mr, l_cleaned, "L (model-ready)")
_preview_clean(n_mr, n_cleaned, "N (model-ready)")

# Save model-ready per-wave
print("\n=== Save model-ready frames ===")
k_mr_path = _safe_to_parquet_or_csv(k_mr, OUT_DIR / "ukhls_k_model_ready", SAVE_FMT)
l_mr_path = _safe_to_parquet_or_csv(l_mr, OUT_DIR / "ukhls_l_model_ready", SAVE_FMT)
n_mr_path = _safe_to_parquet_or_csv(n_mr, OUT_DIR / "ukhls_n_model_ready", SAVE_FMT)

print("\nDone.")
print("Standardized saved ->", k_std_path, l_std_path, n_std_path)
print("Model-ready saved  ->", k_mr_path, l_mr_path, n_mr_path)

# === Promote *_clean -> base name, keep *_missflag (build analysis-ready) ===
def promote_clean_to_base(df: pd.DataFrame, base_cols: list[str]) -> pd.DataFrame:
    """
    For every column in df that has a twin `<col>_clean`:
      - replace the base `<col>` with `<col>_clean` (or create if base missing)
      - drop `<col>_clean`
      - keep `<col>_missflag` as-is
    Returns a frame with canonical names and flags preserved.
    """
    out = df.copy()
    changed = []

    for c in out.columns:
        if c.endswith("_clean"):
            base = c[:-6]  # remove "_clean"
            # move clean values into base
            out[base] = out[c]
            out = out.drop(columns=[c])
            changed.append(base)

    # Order columns: identifiers first, then canonical base_cols, then *_missflag
    id_cols = [col for col in ["pidp", "wave", "wave_num"] if col in out.columns]
    missflags = sorted([c for c in out.columns if c.endswith("_missflag")])

    # Ensure base_cols exist (fill with NA if missing)
    for c in base_cols:
        if c not in out.columns:
            out[c] = pd.NA

    final_order = pd.Index(id_cols + base_cols + missflags).drop_duplicates().tolist()
    out = out.loc[:, final_order]

    print(f"Promoted {len(changed)} columns to cleaned base names: {sorted(changed)}")
    return out


# --- Apply to K/L/N model-ready frames ---
STD_BASE = [
    "ppid","intdaty_dv","age_dv","sex_dv",
    "gor_dv","urban_dv","hhtype_dv","hhsize",
    "ethn_dv","anxiety_raw",
    "jbft_dv","jbnssec_dv","jbhrs",
    "fimnnet_dv","sf12pcs_dv","scghq1_dv","nchild_dv",
    "hrpid","ind5mus_xw","indinui_lw",
]

k_ar = promote_clean_to_base(k_mr, STD_BASE)
l_ar = promote_clean_to_base(l_mr, STD_BASE)
n_ar = promote_clean_to_base(n_mr, STD_BASE)

print("Shapes (analysis-ready) ->",
      "K:", k_ar.shape, "L:", l_ar.shape, "N:", n_ar.shape)

# --- Optional previews
print("\nK (analysis-ready) — 10 rows:"); display(k_ar.head(10))
print("\nL (analysis-ready) — 10 rows:"); display(l_ar.head(10))
print("\nN (analysis-ready) — 10 rows:"); display(n_ar.head(10))

# --- Save analysis-ready frames
k_ar_path = _safe_to_parquet_or_csv(k_ar, OUT_DIR / "ukhls_k_analysis_ready", SAVE_FMT)
l_ar_path = _safe_to_parquet_or_csv(l_ar, OUT_DIR / "ukhls_l_analysis_ready", SAVE_FMT)
n_ar_path = _safe_to_parquet_or_csv(n_ar, OUT_DIR / "ukhls_n_analysis_ready", SAVE_FMT)

print("\nAnalysis-ready saved ->")
print(k_ar_path, "\n", l_ar_path, "\n", n_ar_path)

=== Load saved columns-of-interest ===

=== Standardize per wave ===
K -> BEFORE: (32008, 21) | AFTER (standardized): (32008, 23)
L -> BEFORE: (29271, 21) | AFTER (standardized): (29271, 23)
N -> BEFORE: (35471, 21) | AFTER (standardized): (35471, 23)

K std (10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,68006127.0,68006127.0,68020564.0,0.0,1.703009,2019.0,49.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,26.41,16.0,0.0,K,11
1,68020564.0,68006127.0,68006127.0,0.0,0.000000,2019.0,48.0,1.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,1565.670044,34.68,12.0,0.0,K,11
2,68008847.0,68008847.0,-8.0,0.0,0.794699,2019.0,61.0,2.0,1.0,1.0,...,-8.0,1.0,14.0,39.0,2134.000000,44.20,9.0,0.0,K,11
3,68009527.0,68009527.0,68061288.0,0.0,0.962017,2019.0,41.0,1.0,1.0,1.0,...,-8.0,1.0,15.0,39.0,2043.000000,60.48,16.0,2.0,K,11
4,68061288.0,68009527.0,68009527.0,0.0,0.000000,2019.0,33.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,149.250000,54.23,11.0,2.0,K,11
5,68010887.0,68068082.0,68068082.0,0.0,1.055802,2019.0,55.0,2.0,1.0,1.0,...,-8.0,1.0,25.0,32.0,1250.000000,57.28,9.0,0.0,K,11
6,68068082.0,68068082.0,68010887.0,0.0,0.000000,2019.0,58.0,1.0,1.0,1.0,...,-8.0,1.0,19.0,-8.0,690.859985,56.15,9.0,0.0,K,11
7,68014287.0,68014287.0,-8.0,0.0,0.000000,2019.0,49.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,715.869995,44.37,23.0,1.0,K,11
8,68020407.0,68020407.0,-8.0,0.0,0.833442,2019.0,82.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,1621.670044,18.07,13.0,0.0,K,11
9,68028575.0,68157166.0,68157166.0,0.0,1.029255,2019.0,28.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,56.71,6.0,2.0,K,11



L std (10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,68008847.0,68008847.0,-8.0,0.0,0.739967,2020.0,62.0,2.0,1.0,1.0,...,-8.0,1.0,14.0,39.0,2288.000000,37.58,12.0,0.0,L,12
1,68009527.0,68034180.0,68061288.0,0.0,0.950116,2020.0,43.0,1.0,1.0,1.0,...,-8.0,1.0,15.0,36.5,2060.000000,56.37,11.0,2.0,L,12
2,68061288.0,68034180.0,68009527.0,0.0,0.000000,2020.0,34.0,2.0,1.0,1.0,...,-8.0,2.0,24.0,10.0,474.000000,61.73,15.0,2.0,L,12
3,68010887.0,68010887.0,68068082.0,0.0,0.998976,2020.0,56.0,2.0,1.0,1.0,...,-8.0,1.0,25.0,32.0,1200.000000,51.64,11.0,0.0,L,12
4,68068082.0,68010887.0,68010887.0,0.0,0.000000,2020.0,59.0,1.0,1.0,1.0,...,-8.0,1.0,17.0,-8.0,3275.909912,56.15,10.0,0.0,L,12
5,68028575.0,68095380.0,68157166.0,0.0,1.031810,2020.0,28.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,2.0,L,12
6,68157166.0,68095380.0,68028575.0,0.0,0.000000,2020.0,35.0,1.0,1.0,1.0,...,-8.0,1.0,8.0,38.0,2974.000000,57.47,7.0,2.0,L,12
7,68029927.0,68029939.0,68029931.0,0.0,0.000000,2020.0,48.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,89.699997,53.79,9.0,0.0,L,12
8,68029939.0,68029939.0,-8.0,0.0,0.000000,2020.0,16.0,1.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,0.0,L,12
9,68149808.0,68029939.0,-8.0,0.0,0.000000,2020.0,23.0,2.0,1.0,1.0,...,-8.0,1.0,7.0,30.0,865.000000,56.15,8.0,0.0,L,12



N std (10 rows):


...


L (analysis-ready) — 10 rows:


,pidp,wave,wave_num,ppid,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,hhtype_dv,...,fimnnet_dv_missflag,gor_dv_missflag,jbft_dv_missflag,jbhrs_missflag,jbnssec_dv_missflag,ppid_missflag,scghq1_dv_missflag,sex_dv_missflag,sf12pcs_dv_missflag,urban_dv_missflag
0,68008847.0,L,12,NaN,2020.0,62.0,2.0,1.0,1.0,2.0,...,observed,observed,observed,observed,observed,inapplicable,observed,observed,observed,observed
1,68009527.0,L,12,68061288.0,2020.0,43.0,1.0,1.0,1.0,11.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
2,68061288.0,L,12,68009527.0,2020.0,34.0,2.0,1.0,1.0,11.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
3,68010887.0,L,12,68068082.0,2020.0,56.0,2.0,1.0,1.0,6.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
4,68068082.0,L,12,68010887.0,2020.0,59.0,1.0,1.0,1.0,6.0,...,observed,observed,observed,inapplicable,observed,observed,observed,observed,observed,observed
5,68028575.0,L,12,68157166.0,2020.0,28.0,2.0,1.0,1.0,11.0,...,observed,observed,inapplicable,inapplicable,inapplicable,observed,observed,observed,observed,observed
6,68157166.0,L,12,68028575.0,2020.0,35.0,1.0,1.0,1.0,11.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
7,68029927.0,L,12,68029931.0,2020.0,48.0,2.0,1.0,1.0,20.0,...,observed,observed,inapplicable,inapplicable,inapplicable,observed,observed,observed,observed,observed
8,68029939.0,L,12,NaN,2020.0,16.0,1.0,1.0,1.0,20.0,...,observed,observed,inapplicable,inapplicable,inapplicable,inapplicable,observed,observed,observed,observed
9,68149808.0,L,12,NaN,2020.0,23.0,2.0,1.0,1.0,20.0,...,observed,observed,observed,observed,observed,inapplicable,observed,observed,observed,observed



N (analysis-ready) — 10 rows:


,pidp,wave,wave_num,ppid,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,hhtype_dv,...,ethn_dv_missflag,gor_dv_missflag,jbft_dv_missflag,jbhrs_missflag,jbnssec_dv_missflag,ppid_missflag,scghq1_dv_missflag,sex_dv_missflag,sf12pcs_dv_missflag,urban_dv_missflag
0,22445.0,N,14,277059298.0,2022.0,37.0,2.0,8.0,1.0,11.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
1,29925.0,N,14,NaN,2022.0,45.0,2.0,7.0,1.0,5.0,...,observed,observed,observed,observed,observed,inapplicable,observed,observed,observed,observed
2,76165.0,N,14,142378492.0,2022.0,39.0,2.0,5.0,1.0,11.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
3,280165.0,N,14,756200970.0,2022.0,43.0,2.0,8.0,2.0,20.0,...,observed,observed,inapplicable,inapplicable,inapplicable,observed,observed,observed,observed,observed
4,469205.0,N,14,NaN,2022.0,32.0,2.0,4.0,1.0,5.0,...,observed,observed,observed,observed,observed,inapplicable,observed,observed,observed,observed
5,599765.0,N,14,210167702.0,2022.0,35.0,2.0,5.0,1.0,10.0,...,observed,observed,observed,observed,observed,observed,observed,observed,observed,observed
6,732365.0,N,14,NaN,2022.0,37.0,1.0,2.0,1.0,19.0,...,observed,observed,inapplicable,inapplicable,inapplicable,inapplicable,observed,observed,observed,observed
7,1587125.0,N,14,NaN,2022.0,56.0,2.0,1.0,1.0,3.0,...,observed,observed,observed,observed,observed,inapplicable,observed,observed,observed,observed
8,2888645.0,N,14,NaN,2022.0,33.0,2.0,11.0,1.0,3.0,...,observed,observed,observed,observed,observed,inapplicable,observed,observed,observed,observed
9,3424485.0,N,14,NaN,2022.0,86.0,2.0,6.0,2.0,20.0,...,observed,observed,inapplicable,inapplicable,inapplicable,inapplicable,observed,observed,observed,observed


Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_k_analysis_ready.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_l_analysis_ready.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_n_analysis_ready.parquet

Analysis-ready saved ->
C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_k_analysis_ready.parquet 
 C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_l_analysis_ready.parquet 
 C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_n_analysis_ready.parquet


In [ ]:
# ---------- Keep only *_clean and *_missflag for analysis (drop raw) ----------
# Assumes previous script created k_mr, l_mr, n_mr and OUT_DIR, SAVE_FMT, _safe_to_parquet_or_csv

# Ensure MISS_LABELS exists (kept for clarity — already present upstream)
MISS_LABELS = {
    -9: "missing",
    -8: "inapplicable",
    -7: "proxy/partial",
    -2: "refusal",
    -1: "don't know",
}
MISS_CODES = set(MISS_LABELS.keys())

def keep_clean_and_flags(df: pd.DataFrame, keep_core=("pidp", "wave", "wave_num")) -> pd.DataFrame:
    """
    Return a frame that keeps only:
      - core identifier columns (pidp, wave, wave_num)
      - all *_clean columns
      - all *_missflag columns
    Also reorders so that for each base var the _clean and _missflag appear together.
    """
    # which cols to keep
    keep = [c for c in df.columns if c in keep_core or c.endswith("_clean") or c.endswith("_missflag")]
    df2 = df.loc[:, [c for c in keep if c in df.columns]].copy()

    # Build ordered list: core first, then for each base var <base>_clean + <base>_missflag if present
    ordered = [c for c in keep_core if c in df2.columns]
    # collect unique bases from *_clean columns in original order
    clean_cols = [c for c in df2.columns if c.endswith("_clean")]
    for clean in clean_cols:
        base = clean[:-6]  # strip "_clean"
        ordered.append(f"{base}_clean")
        flag = f"{base}_missflag"
        if flag in df2.columns:
            ordered.append(flag)

    # There may be some *_missflag without a _clean (rare). Append any remaining missflags not included.
    remaining_flags = [c for c in df2.columns if c.endswith("_missflag") and c not in ordered]
    ordered.extend(remaining_flags)

    # Finally, ensure we only include columns that exist and deduplicate preserving order
    seen = set()
    final_order = []
    for c in ordered:
        if c in df2.columns and c not in seen:
            final_order.append(c)
            seen.add(c)

    # Reindex with the final order
    return df2.reindex(columns=final_order)

# Apply to each wave
k_analysis = keep_clean_and_flags(k_mr)
l_analysis = keep_clean_and_flags(l_mr)
n_analysis = keep_clean_and_flags(n_mr)

# Print shapes before/after (use original model-ready frames to compare)
print("Shapes (model-ready) -> K:", k_mr.shape, "L:", l_mr.shape, "N:", n_mr.shape)
print("Shapes (analysis-ready: keep only _clean/_missflag) -> K:", k_analysis.shape,
      "L:", l_analysis.shape, "N:", n_analysis.shape)

# Quick preview (10 rows each)
print("\nK (analysis-ready) — first 10 rows:")
display(k_analysis.head(10))
print("\nL (analysis-ready) — first 10 rows:")
display(l_analysis.head(10))
print("\nN (analysis-ready) — first 10 rows:")
display(n_analysis.head(10))

# Save analysis-ready frames (Parquet preferred; will fall back to CSV via your helper)
_safe_to_parquet_or_csv(k_analysis, OUT_DIR / "ukhls_k_analysis_ready", SAVE_FMT)
_safe_to_parquet_or_csv(l_analysis, OUT_DIR / "ukhls_l_analysis_ready", SAVE_FMT)
_safe_to_parquet_or_csv(n_analysis, OUT_DIR / "ukhls_n_analysis_ready", SAVE_FMT)
  
print("\nSaved analysis-ready files to:", OUT_DIR)

Shapes (model-ready) -> K: (32008, 51) L: (29271, 47) N: (35471, 47)
Shapes (analysis-ready: keep only _clean/_missflag) -> K: (32008, 31) L: (29271, 27) N: (35471, 27)

K (analysis-ready) — first 10 rows:


,pidp,wave,wave_num,ppid_clean,ppid_missflag,intdaty_dv_clean,intdaty_dv_missflag,age_dv_clean,age_dv_missflag,sex_dv_clean,...,jbnssec_dv_clean,jbnssec_dv_missflag,jbhrs_clean,jbhrs_missflag,fimnnet_dv_clean,fimnnet_dv_missflag,sf12pcs_dv_clean,sf12pcs_dv_missflag,scghq1_dv_clean,scghq1_dv_missflag
0,68006127.0,K,11,68020564.0,observed,2019.0,observed,49.0,observed,2.0,...,NaN,inapplicable,NaN,inapplicable,0.000000,observed,26.41,observed,16.0,observed
1,68020564.0,K,11,68006127.0,observed,2019.0,observed,48.0,observed,1.0,...,NaN,inapplicable,NaN,inapplicable,1565.670044,observed,34.68,observed,12.0,observed
2,68008847.0,K,11,NaN,inapplicable,2019.0,observed,61.0,observed,2.0,...,14.0,observed,39.0,observed,2134.000000,observed,44.20,observed,9.0,observed
3,68009527.0,K,11,68061288.0,observed,2019.0,observed,41.0,observed,1.0,...,15.0,observed,39.0,observed,2043.000000,observed,60.48,observed,16.0,observed
4,68061288.0,K,11,68009527.0,observed,2019.0,observed,33.0,observed,2.0,...,NaN,inapplicable,NaN,inapplicable,149.250000,observed,54.23,observed,11.0,observed
5,68010887.0,K,11,68068082.0,observed,2019.0,observed,55.0,observed,2.0,...,25.0,observed,32.0,observed,1250.000000,observed,57.28,observed,9.0,observed
6,68068082.0,K,11,68010887.0,observed,2019.0,observed,58.0,observed,1.0,...,19.0,observed,NaN,inapplicable,690.859985,observed,56.15,observed,9.0,observed
7,68014287.0,K,11,NaN,inapplicable,2019.0,observed,49.0,observed,2.0,...,NaN,inapplicable,NaN,inapplicable,715.869995,observed,44.37,observed,23.0,observed
8,68020407.0,K,11,NaN,inapplicable,2019.0,observed,82.0,observed,2.0,...,NaN,inapplicable,NaN,inapplicable,1621.670044,observed,18.07,observed,13.0,observed
9,68028575.0,K,11,68157166.0,observed,2019.0,observed,28.0,observed,2.0,...,NaN,inapplicable,NaN,inapplicable,0.000000,observed,56.71,observed,6.0,observed



L (analysis-ready) — first 10 rows:


,pidp,wave,wave_num,ppid_clean,ppid_missflag,sex_dv_clean,sex_dv_missflag,gor_dv_clean,gor_dv_missflag,urban_dv_clean,...,jbnssec_dv_clean,jbnssec_dv_missflag,jbhrs_clean,jbhrs_missflag,fimnnet_dv_clean,fimnnet_dv_missflag,sf12pcs_dv_clean,sf12pcs_dv_missflag,scghq1_dv_clean,scghq1_dv_missflag
0,68008847.0,L,12,NaN,inapplicable,2.0,observed,1.0,observed,1.0,...,14.0,observed,39.0,observed,2288.000000,observed,37.58,observed,12.0,observed
1,68009527.0,L,12,68061288.0,observed,1.0,observed,1.0,observed,1.0,...,15.0,observed,36.5,observed,2060.000000,observed,56.37,observed,11.0,observed
2,68061288.0,L,12,68009527.0,observed,2.0,observed,1.0,observed,1.0,...,24.0,observed,10.0,observed,474.000000,observed,61.73,observed,15.0,observed
3,68010887.0,L,12,68068082.0,observed,2.0,observed,1.0,observed,1.0,...,25.0,observed,32.0,observed,1200.000000,observed,51.64,observed,11.0,observed
4,68068082.0,L,12,68010887.0,observed,1.0,observed,1.0,observed,1.0,...,17.0,observed,NaN,inapplicable,3275.909912,observed,56.15,observed,10.0,observed
5,68028575.0,L,12,68157166.0,observed,2.0,observed,1.0,observed,1.0,...,NaN,inapplicable,NaN,inapplicable,0.000000,observed,57.76,observed,8.0,observed
6,68157166.0,L,12,68028575.0,observed,1.0,observed,1.0,observed,1.0,...,8.0,observed,38.0,observed,2974.000000,observed,57.47,observed,7.0,observed
7,68029927.0,L,12,68029931.0,observed,2.0,observed,1.0,observed,1.0,...,NaN,inapplicable,NaN,inapplicable,89.699997,observed,53.79,observed,9.0,observed
8,68029939.0,L,12,NaN,inapplicable,1.0,observed,1.0,observed,1.0,...,NaN,inapplicable,NaN,inapplicable,0.000000,observed,57.76,observed,8.0,observed
9,68149808.0,L,12,NaN,inapplicable,2.0,observed,1.0,observed,1.0,...,7.0,observed,30.0,observed,865.000000,observed,56.15,observed,8.0,observed



N (analysis-ready) — first 10 rows:


,pidp,wave,wave_num,ppid_clean,ppid_missflag,age_dv_clean,age_dv_missflag,sex_dv_clean,sex_dv_missflag,gor_dv_clean,...,jbft_dv_clean,jbft_dv_missflag,jbnssec_dv_clean,jbnssec_dv_missflag,jbhrs_clean,jbhrs_missflag,sf12pcs_dv_clean,sf12pcs_dv_missflag,scghq1_dv_clean,scghq1_dv_missflag
0,22445.0,N,14,277059298.0,observed,37.0,observed,2.0,observed,8.0,...,1.0,observed,2.0,observed,28.0,observed,62.83,observed,24.0,observed
1,29925.0,N,14,NaN,inapplicable,45.0,observed,2.0,observed,7.0,...,2.0,observed,14.0,observed,29.0,observed,65.47,observed,23.0,observed
2,76165.0,N,14,142378492.0,observed,39.0,observed,2.0,observed,5.0,...,1.0,observed,11.0,observed,35.0,observed,57.20,observed,12.0,observed
3,280165.0,N,14,756200970.0,observed,43.0,observed,2.0,observed,8.0,...,NaN,inapplicable,NaN,inapplicable,NaN,inapplicable,58.55,observed,16.0,observed
4,469205.0,N,14,NaN,inapplicable,32.0,observed,2.0,observed,4.0,...,2.0,observed,25.0,observed,16.0,observed,49.93,observed,16.0,observed
5,599765.0,N,14,210167702.0,observed,35.0,observed,2.0,observed,5.0,...,1.0,observed,2.0,observed,37.0,observed,56.15,observed,6.0,observed
6,732365.0,N,14,NaN,inapplicable,37.0,observed,1.0,observed,2.0,...,NaN,inapplicable,NaN,inapplicable,NaN,inapplicable,45.49,observed,35.0,observed
7,1587125.0,N,14,NaN,inapplicable,56.0,observed,2.0,observed,1.0,...,1.0,observed,7.0,observed,37.0,observed,37.37,observed,15.0,observed
8,2888645.0,N,14,NaN,inapplicable,33.0,observed,2.0,observed,11.0,...,1.0,observed,2.0,observed,38.0,observed,56.15,observed,6.0,observed
9,3424485.0,N,14,NaN,inapplicable,86.0,observed,2.0,observed,6.0,...,NaN,inapplicable,NaN,inapplicable,NaN,inapplicable,33.23,observed,10.0,observed


Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_k_analysis_ready.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_l_analysis_ready.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_n_analysis_ready.parquet

Saved analysis-ready files to: C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed


In [22]:
# === Build an "analysis view": keep clean values + missflags, drop raw duplicates ===
import pandas as pd

def compress_for_analysis(df: pd.DataFrame,
                          keep_flags="all",   # "all" | "none" | list of base names to keep flags for
                          rename_clean=True   # rename <col>_clean -> <col>
                          ) -> pd.DataFrame:
    """
    Returns a frame where:
      - For every base column that has <base>_clean, we keep the clean version
        (optionally renamed back to <base>) and drop the raw <base>.
      - Missflag columns are kept per keep_flags.
      - Columns with no _clean counterpart are left as-is.
      - Deduplicates any accidental duplicate column names.
    """
    df2 = df.copy()

    # 1) identify base columns with a clean twin
    clean_cols = [c for c in df2.columns if c.endswith("_clean")]
    bases = [c[:-6] for c in clean_cols]  # strip "_clean"

    # 2) rename clean -> base and drop raw (or keep clean name if rename_clean=False)
    for base in bases:
        clean = f"{base}_clean"
        if rename_clean:
            # drop raw base if present and not the same object
            if base in df2.columns:
                df2 = df2.drop(columns=[base])
            df2 = df2.rename(columns={clean: base})
        else:
            # keep both, just keep the clean twin named *_clean
            pass

    # 3) handle missflags
    flag_cols = [f"{b}_missflag" for b in bases if f"{b}_missflag" in df2.columns]
    if keep_flags == "none":
        df2 = df2.drop(columns=flag_cols, errors="ignore")
    elif isinstance(keep_flags, (list, tuple, set)):
        keep_list = {f"{b}_missflag" for b in keep_flags}
        drop_list = [c for c in flag_cols if c not in keep_list]
        df2 = df2.drop(columns=drop_list, errors="ignore")
    # else: "all" -> keep all flag_cols

    # 4) deduplicate any repeated column names (e.g., accidental duplicates)
    df2 = df2.loc[:, ~df2.columns.duplicated(keep="first")]

    return df2

# --- Example: create compact analysis tables for each wave ---
# (k_mr, l_mr, n_mr are your model-ready frames with *_clean + *_missflag)

# Option A: keep all missflags
k_analysis = compress_for_analysis(k_mr, keep_flags="all", rename_clean=True)
l_analysis = compress_for_analysis(l_mr, keep_flags="all", rename_clean=True)
n_analysis = compress_for_analysis(n_mr, keep_flags="all", rename_clean=True)

print("K analysis shape:", k_analysis.shape)
print("L analysis shape:", l_analysis.shape)
print("N analysis shape:", n_analysis.shape)

# Optionally save the analysis view (Parquet with CSV fallback if you have that helper available)
# _safe_to_parquet_or_csv(k_analysis, OUT_DIR / "ukhls_k_analysis", SAVE_FMT)
# _safe_to_parquet_or_csv(l_analysis, OUT_DIR / "ukhls_l_analysis", SAVE_FMT)
# _safe_to_parquet_or_csv(n_analysis, OUT_DIR / "ukhls_n_analysis", SAVE_FMT)

# ---- Quick sanity previews (10 rows each) from SAVED files only ----
k_sel = _load_saved("k")
l_sel = _load_saved("l")
n_sel = _load_saved("n")
print("K — preview (10 rows):")
display(k_sel[k_cols_interest].head(15))
print("L — preview (10 rows):")
display(l_sel[l_cols_interest].head(15))
print("N — preview (10 rows):")
display(n_sel[n_cols_interest].head(15))

K analysis shape: (32008, 37)
L analysis shape: (29271, 35)
N analysis shape: (35471, 35)
K — preview (10 rows):


,pidp,k_hrpid,k_ppid,k_ind5mus_xw,k_indinui_lw,k_intdaty_dv,k_age_dv,k_sex_dv,k_gor_dv,k_urban_dv,...,k_hhsize,k_ethn_dv,k_mhealthtyp1,k_jbft_dv,k_jbnssec_dv,k_jbhrs,k_fimnnet_dv,k_sf12pcs_dv,k_scghq1_dv,k_nchild_dv
0,68006127.0,68006127.0,68020564.0,0.0,1.703009,2019.0,49.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,26.41,16.0,0.0
1,68020564.0,68006127.0,68006127.0,0.0,0.000000,2019.0,48.0,1.0,1.0,1.0,...,2.0,1.0,-8.0,-8.0,-8.0,-8.0,1565.670044,34.68,12.0,0.0
2,68008847.0,68008847.0,-8.0,0.0,0.794699,2019.0,61.0,2.0,1.0,1.0,...,1.0,1.0,-8.0,1.0,14.0,39.0,2134.000000,44.20,9.0,0.0
3,68009527.0,68009527.0,68061288.0,0.0,0.962017,2019.0,41.0,1.0,1.0,1.0,...,4.0,1.0,-8.0,1.0,15.0,39.0,2043.000000,60.48,16.0,2.0
4,68061288.0,68009527.0,68009527.0,0.0,0.000000,2019.0,33.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,-8.0,-8.0,-8.0,149.250000,54.23,11.0,2.0
5,68010887.0,68068082.0,68068082.0,0.0,1.055802,2019.0,55.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,25.0,32.0,1250.000000,57.28,9.0,0.0
6,68068082.0,68068082.0,68010887.0,0.0,0.000000,2019.0,58.0,1.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,19.0,-8.0,690.859985,56.15,9.0,0.0
7,68014287.0,68014287.0,-8.0,0.0,0.000000,2019.0,49.0,2.0,1.0,1.0,...,3.0,1.0,-8.0,-8.0,-8.0,-8.0,715.869995,44.37,23.0,1.0
8,68020407.0,68020407.0,-8.0,0.0,0.833442,2019.0,82.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,-8.0,-8.0,-8.0,1621.670044,18.07,13.0,0.0
9,68028575.0,68157166.0,68157166.0,0.0,1.029255,2019.0,28.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,56.71,6.0,2.0


L — preview (10 rows):


,pidp,l_hrpid,l_ppid,l_ind5mus_xw,l_indinui_lw,l_intdaty_dv,l_age_dv,l_sex_dv,l_gor_dv,l_urban_dv,...,l_hhsize,l_ethn_dv,l_mhealthtyp1,l_jbft_dv,l_jbnssec_dv,l_jbhrs,l_fimnnet_dv,l_sf12pcs_dv,l_scghq1_dv,l_nchild_dv
0,68008847.0,68008847.0,-8.0,0.0,0.739967,2020.0,62.0,2.0,1.0,1.0,...,1.0,1.0,-8.0,1.0,14.0,39.0,2288.000000,37.58,12.0,0.0
1,68009527.0,68034180.0,68061288.0,0.0,0.950116,2020.0,43.0,1.0,1.0,1.0,...,4.0,1.0,-8.0,1.0,15.0,36.5,2060.000000,56.37,11.0,2.0
2,68061288.0,68034180.0,68009527.0,0.0,0.000000,2020.0,34.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,2.0,24.0,10.0,474.000000,61.73,15.0,2.0
3,68010887.0,68010887.0,68068082.0,0.0,0.998976,2020.0,56.0,2.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,25.0,32.0,1200.000000,51.64,11.0,0.0
4,68068082.0,68010887.0,68010887.0,0.0,0.000000,2020.0,59.0,1.0,1.0,1.0,...,2.0,1.0,-8.0,1.0,17.0,-8.0,3275.909912,56.15,10.0,0.0
5,68028575.0,68095380.0,68157166.0,0.0,1.031810,2020.0,28.0,2.0,1.0,1.0,...,4.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,2.0
6,68157166.0,68095380.0,68028575.0,0.0,0.000000,2020.0,35.0,1.0,1.0,1.0,...,4.0,1.0,-8.0,1.0,8.0,38.0,2974.000000,57.47,7.0,2.0
7,68029927.0,68029939.0,68029931.0,0.0,0.000000,2020.0,48.0,2.0,1.0,1.0,...,5.0,1.0,-8.0,-8.0,-8.0,-8.0,89.699997,53.79,9.0,0.0
8,68029939.0,68029939.0,-8.0,0.0,0.000000,2020.0,16.0,1.0,1.0,1.0,...,5.0,1.0,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,0.0
9,68149808.0,68029939.0,-8.0,0.0,0.000000,2020.0,23.0,2.0,1.0,1.0,...,5.0,1.0,-8.0,1.0,7.0,30.0,865.000000,56.15,8.0,0.0


N — preview (10 rows):


,pidp,n_hrpid,n_ppid,n_ind5mus_xw,n_indinui_lw,n_intdaty_dv,n_age_dv,n_sex_dv,n_gor_dv,n_urban_dv,...,n_hhsize,n_ethn_dv,n_mhgad,n_jbft_dv,n_jbnssec_dv,n_jbhrs,n_fimnnet_dv,n_sf12pcs_dv,n_scghq1_dv,n_nchild_dv
0,22445.0,276841780.0,277059298.0,0.0,0.0,2022.0,37.0,2.0,8.0,1.0,...,4.0,1.0,2.0,1.0,2.0,28.0,1857.079956,62.83,24.0,2.0
1,29925.0,622866606.0,-8.0,0.0,0.0,2022.0,45.0,2.0,7.0,1.0,...,3.0,1.0,2.0,2.0,14.0,29.0,2378.750000,65.47,23.0,2.0
2,76165.0,141045780.0,142378492.0,0.0,0.0,2022.0,39.0,2.0,5.0,1.0,...,4.0,1.0,2.0,1.0,11.0,35.0,3206.000000,57.20,12.0,2.0
3,280165.0,783876922.0,756200970.0,0.0,0.0,2022.0,43.0,2.0,8.0,2.0,...,4.0,1.0,2.0,-8.0,-8.0,-8.0,94.470001,58.55,16.0,1.0
4,469205.0,414412580.0,-8.0,0.0,0.0,2022.0,32.0,2.0,4.0,1.0,...,3.0,1.0,2.0,2.0,25.0,16.0,2056.080078,49.93,16.0,2.0
5,599765.0,209943344.0,210167702.0,0.0,0.0,2022.0,35.0,2.0,5.0,1.0,...,3.0,1.0,2.0,1.0,2.0,37.0,2839.649902,56.15,6.0,1.0
6,732365.0,732365.0,-8.0,0.0,0.0,2022.0,37.0,1.0,2.0,1.0,...,3.0,1.0,1.0,-8.0,-8.0,-8.0,838.500000,45.49,35.0,0.0
7,1587125.0,1587125.0,-8.0,0.0,0.0,2022.0,56.0,2.0,1.0,1.0,...,1.0,1.0,2.0,1.0,7.0,37.0,2290.000000,37.37,15.0,0.0
8,2888645.0,2888645.0,-8.0,0.0,0.0,2022.0,33.0,2.0,11.0,1.0,...,1.0,1.0,2.0,1.0,2.0,38.0,2300.000000,56.15,6.0,0.0
9,3424485.0,103550562.0,-8.0,0.0,0.0,2022.0,86.0,2.0,6.0,2.0,...,5.0,1.0,2.0,-8.0,-8.0,-8.0,1092.750000,33.23,10.0,0.0


In [16]:
# === Standardize per-wave columns (from saved columns-of-interest) and save (Parquet/CSV) ===
import pandas as pd
from pathlib import Path

# ---- Missing-code map (kept here for downstream consistency) ----
MISS_LABELS = {
    -9: "missing",
    -8: "inapplicable",
    -7: "proxy/partial",
    -2: "refusal",
    -1: "don't know",
}
MISS_CODES = set(MISS_LABELS.keys())

# ---------- Config ----------
PROJECT_ROOT = Path(PROJECT_ROOT)  # keep your existing var
IN_DIR  = PROJECT_ROOT / "data" / "interim"      # where earlier 'columns_of_interest' files live
OUT_DIR = PROJECT_ROOT / "data" / "processed"    # where standardized outputs will be written
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Choose output format: "parquet" (preferred) or "csv"
SAVE_FMT = "parquet"

# Wave → wave number map
WAVE_NUM = {"k": 11, "l": 12, "n": 14}

# The unified (standardized) variable list we want in every wave (21 core vars)
STD_COLS = [
    "pidp",
    "hrpid", "ppid", "ind5mus_xw", "indinui_lw",
    "intdaty_dv", "age_dv", "sex_dv",
    "gor_dv", "urban_dv", "hhtype_dv", "hhsize",
    "ethn_dv", "anxiety_raw",
    "jbft_dv", "jbnssec_dv", "jbhrs",
    "fimnnet_dv", "sf12pcs_dv", "scghq1_dv", "nchild_dv",
]

# ---------- Helpers ----------
def _find_input_file(prefix: str) -> Path:
    base = IN_DIR / f"ukhls_{prefix}_columns_of_interest"
    pq = base.with_suffix(".parquet")
    cs = base.with_suffix(".csv")
    if pq.exists():
        return pq
    if cs.exists():
        return cs
    raise FileNotFoundError(f"No input found for wave {prefix}: {pq.name} or {cs.name}")

def _load_df(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".parquet":
        return pd.read_parquet(path)
    elif path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    else:
        raise ValueError(f"Unsupported file type: {path}")

def _safe_to_parquet_or_csv(df: pd.DataFrame, path_base: Path, fmt: str = "parquet") -> Path:
    """
    Try Parquet, fall back to CSV on any Arrow issue.
    Returns the final written path.
    """
    df2 = df.copy()
    # Arrow is picky about some extension dtypes; normalize objects/categoricals to string
    for c in df2.columns:
        if isinstance(df2[c].dtype, pd.CategoricalDtype):
            df2[c] = df2[c].astype("string")
        elif pd.api.types.is_object_dtype(df2[c]):
            df2[c] = df2[c].astype("string")

    if fmt.lower() == "parquet":
        try:
            out_path = path_base.with_suffix(".parquet")
            df2.to_parquet(out_path, index=False)
            print(f"Saved Parquet -> {out_path}")
            return out_path
        except Exception as e:
            print(f"Parquet failed ({type(e).__name__}: {e}). Falling back to CSV.")
            out_path = path_base.with_suffix(".csv")
            df.to_csv(out_path, index=False)
            print(f"Saved CSV -> {out_path}")
            return out_path
    elif fmt.lower() == "csv":
        out_path = path_base.with_suffix(".csv")
        df.to_csv(out_path, index=False)
        print(f"Saved CSV -> {out_path}")
        return out_path
    else:
        raise ValueError("SAVE_FMT must be 'parquet' or 'csv'")

def _standardize_wave(df_in: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """
    - Drop wave prefix (k_/l_/n_) from column names
    - Build unified anxiety_raw: use 'mhgad' if present, else 'mhealthtyp1' if present
    - Add wave + wave_num
    - Ensure the standardized 21 columns are present (create missing as NA)
    - Return only STD_COLS + ['wave','wave_num'] in that order
    """
    df = df_in.copy()
    before_shape = df.shape

    # Strip wave prefix on any column that has it
    pref = f"{prefix}_"
    rename_map = {c: c[len(pref):] for c in df.columns if c.startswith(pref)}
    df = df.rename(columns=rename_map)

    # Unified anxiety proxy
    if "mhgad" in df.columns:
        df["anxiety_raw"] = df["mhgad"]
    elif "mhealthtyp1" in df.columns:
        df["anxiety_raw"] = df["mhealthtyp1"]

    # Wave labels
    df["wave"] = prefix.upper()
    df["wave_num"] = WAVE_NUM[prefix]

    # Ensure all standardized columns exist
    for c in STD_COLS:
        if c not in df.columns:
            df[c] = pd.NA

    # Final order: 21 std vars + wave + wave_num
    df = df[STD_COLS + ["wave", "wave_num"]]

    after_shape = df.shape
    print(f"{prefix.upper()} -> BEFORE: {before_shape} | AFTER (standardized): {after_shape}")
    return df

# ---------- Run for K, L, N ----------
print("=== Loading saved per-wave columns-of-interest and standardizing ===")
k_path = _find_input_file("k")
l_path = _find_input_file("l")
n_path = _find_input_file("n")

k_in = _load_df(k_path)
l_in = _load_df(l_path)
n_in = _load_df(n_path)

k_std = _standardize_wave(k_in, "k")
l_std = _standardize_wave(l_in, "l")
n_std = _standardize_wave(n_in, "n")

# Optional: preview
print("\nK (first 10 rows):"); display(k_std.head(10))
print("\nL (first 10 rows):"); display(l_std.head(10))
print("\nN (first 10 rows):"); display(n_std.head(10))

# ---------- Save standardized per-wave frames ----------
_safe_to_parquet_or_csv(k_std, OUT_DIR / "ukhls_k_standardized", SAVE_FMT)
_safe_to_parquet_or_csv(l_std, OUT_DIR / "ukhls_l_standardized", SAVE_FMT)
_safe_to_parquet_or_csv(n_std, OUT_DIR / "ukhls_n_standardized", SAVE_FMT)

print("\nDone. Standardized files saved in:", OUT_DIR)


=== Loading saved per-wave columns-of-interest and standardizing ===
K -> BEFORE: (32008, 21) | AFTER (standardized): (32008, 23)
L -> BEFORE: (29271, 21) | AFTER (standardized): (29271, 23)
N -> BEFORE: (35471, 21) | AFTER (standardized): (35471, 23)

K (first 10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,68006127.0,68006127.0,68020564.0,0.0,1.703009,2019.0,49.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,26.41,16.0,0.0,K,11
1,68020564.0,68006127.0,68006127.0,0.0,0.000000,2019.0,48.0,1.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,1565.670044,34.68,12.0,0.0,K,11
2,68008847.0,68008847.0,-8.0,0.0,0.794699,2019.0,61.0,2.0,1.0,1.0,...,-8.0,1.0,14.0,39.0,2134.000000,44.20,9.0,0.0,K,11
3,68009527.0,68009527.0,68061288.0,0.0,0.962017,2019.0,41.0,1.0,1.0,1.0,...,-8.0,1.0,15.0,39.0,2043.000000,60.48,16.0,2.0,K,11
4,68061288.0,68009527.0,68009527.0,0.0,0.000000,2019.0,33.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,149.250000,54.23,11.0,2.0,K,11
5,68010887.0,68068082.0,68068082.0,0.0,1.055802,2019.0,55.0,2.0,1.0,1.0,...,-8.0,1.0,25.0,32.0,1250.000000,57.28,9.0,0.0,K,11
6,68068082.0,68068082.0,68010887.0,0.0,0.000000,2019.0,58.0,1.0,1.0,1.0,...,-8.0,1.0,19.0,-8.0,690.859985,56.15,9.0,0.0,K,11
7,68014287.0,68014287.0,-8.0,0.0,0.000000,2019.0,49.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,715.869995,44.37,23.0,1.0,K,11
8,68020407.0,68020407.0,-8.0,0.0,0.833442,2019.0,82.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,1621.670044,18.07,13.0,0.0,K,11
9,68028575.0,68157166.0,68157166.0,0.0,1.029255,2019.0,28.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,56.71,6.0,2.0,K,11



L (first 10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,68008847.0,68008847.0,-8.0,0.0,0.739967,2020.0,62.0,2.0,1.0,1.0,...,-8.0,1.0,14.0,39.0,2288.000000,37.58,12.0,0.0,L,12
1,68009527.0,68034180.0,68061288.0,0.0,0.950116,2020.0,43.0,1.0,1.0,1.0,...,-8.0,1.0,15.0,36.5,2060.000000,56.37,11.0,2.0,L,12
2,68061288.0,68034180.0,68009527.0,0.0,0.000000,2020.0,34.0,2.0,1.0,1.0,...,-8.0,2.0,24.0,10.0,474.000000,61.73,15.0,2.0,L,12
3,68010887.0,68010887.0,68068082.0,0.0,0.998976,2020.0,56.0,2.0,1.0,1.0,...,-8.0,1.0,25.0,32.0,1200.000000,51.64,11.0,0.0,L,12
4,68068082.0,68010887.0,68010887.0,0.0,0.000000,2020.0,59.0,1.0,1.0,1.0,...,-8.0,1.0,17.0,-8.0,3275.909912,56.15,10.0,0.0,L,12
5,68028575.0,68095380.0,68157166.0,0.0,1.031810,2020.0,28.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,2.0,L,12
6,68157166.0,68095380.0,68028575.0,0.0,0.000000,2020.0,35.0,1.0,1.0,1.0,...,-8.0,1.0,8.0,38.0,2974.000000,57.47,7.0,2.0,L,12
7,68029927.0,68029939.0,68029931.0,0.0,0.000000,2020.0,48.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,89.699997,53.79,9.0,0.0,L,12
8,68029939.0,68029939.0,-8.0,0.0,0.000000,2020.0,16.0,1.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,0.0,L,12
9,68149808.0,68029939.0,-8.0,0.0,0.000000,2020.0,23.0,2.0,1.0,1.0,...,-8.0,1.0,7.0,30.0,865.000000,56.15,8.0,0.0,L,12



N (first 10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,22445.0,276841780.0,277059298.0,0.0,0.0,2022.0,37.0,2.0,8.0,1.0,...,2.0,1.0,2.0,28.0,1857.079956,62.83,24.0,2.0,N,14
1,29925.0,622866606.0,-8.0,0.0,0.0,2022.0,45.0,2.0,7.0,1.0,...,2.0,2.0,14.0,29.0,2378.750000,65.47,23.0,2.0,N,14
2,76165.0,141045780.0,142378492.0,0.0,0.0,2022.0,39.0,2.0,5.0,1.0,...,2.0,1.0,11.0,35.0,3206.000000,57.20,12.0,2.0,N,14
3,280165.0,783876922.0,756200970.0,0.0,0.0,2022.0,43.0,2.0,8.0,2.0,...,2.0,-8.0,-8.0,-8.0,94.470001,58.55,16.0,1.0,N,14
4,469205.0,414412580.0,-8.0,0.0,0.0,2022.0,32.0,2.0,4.0,1.0,...,2.0,2.0,25.0,16.0,2056.080078,49.93,16.0,2.0,N,14
5,599765.0,209943344.0,210167702.0,0.0,0.0,2022.0,35.0,2.0,5.0,1.0,...,2.0,1.0,2.0,37.0,2839.649902,56.15,6.0,1.0,N,14
6,732365.0,732365.0,-8.0,0.0,0.0,2022.0,37.0,1.0,2.0,1.0,...,1.0,-8.0,-8.0,-8.0,838.500000,45.49,35.0,0.0,N,14
7,1587125.0,1587125.0,-8.0,0.0,0.0,2022.0,56.0,2.0,1.0,1.0,...,2.0,1.0,7.0,37.0,2290.000000,37.37,15.0,0.0,N,14
8,2888645.0,2888645.0,-8.0,0.0,0.0,2022.0,33.0,2.0,11.0,1.0,...,2.0,1.0,2.0,38.0,2300.000000,56.15,6.0,0.0,N,14
9,3424485.0,103550562.0,-8.0,0.0,0.0,2022.0,86.0,2.0,6.0,2.0,...,2.0,-8.0,-8.0,-8.0,1092.750000,33.23,10.0,0.0,N,14


Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_k_standardized.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_l_standardized.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_n_standardized.parquet

Done. Standardized files saved in: C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed


In [19]:
# === Standardize per-wave columns (from saved columns-of-interest) and save (Parquet/CSV) ===
import pandas as pd
from pathlib import Path

# ---------- Config ----------
PROJECT_ROOT = Path(PROJECT_ROOT)  # keep your existing var
IN_DIR  = PROJECT_ROOT / "data" / "interim"      # where earlier 'columns_of_interest' files live
OUT_DIR = PROJECT_ROOT / "data" / "processed"    # where standardized outputs will be written
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Choose output format: "parquet" (preferred) or "csv"
SAVE_FMT = "parquet"

# Wave → wave number map
WAVE_NUM = {"k": 11, "l": 12, "n": 14}

# The unified (standardized) variable list we want in every wave (21 core vars)
STD_COLS = [
    "pidp",
    "hrpid", "ppid", "ind5mus_xw", "indinui_lw",
    "intdaty_dv", "age_dv", "sex_dv",
    "gor_dv", "urban_dv", "hhtype_dv", "hhsize",
    "ethn_dv", "anxiety_raw",
    "jbft_dv", "jbnssec_dv", "jbhrs",
    "fimnnet_dv", "sf12pcs_dv", "scghq1_dv", "nchild_dv",
]

# ---------- Helpers ----------
def _find_input_file(prefix: str) -> Path:
    """
    Find the saved per-wave 'columns_of_interest' file for the given prefix (k/l/n),
    preferring Parquet if present, otherwise CSV.
    """
    base = IN_DIR / f"ukhls_{prefix}_columns_of_interest"
    pq = base.with_suffix(".parquet")
    cs = base.with_suffix(".csv")
    if pq.exists():
        return pq
    if cs.exists():
        return cs
    raise FileNotFoundError(f"No input found for wave {prefix}: {pq.name} or {cs.name}")

def _load_df(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".parquet":
        return pd.read_parquet(path)
    elif path.suffix.lower() == ".csv":
        return pd.read_csv(path)
    else:
        raise ValueError(f"Unsupported file type: {path}")

def _safe_to_parquet_or_csv(df: pd.DataFrame, path_base: Path, fmt: str = "parquet") -> Path:
    """
    Try Parquet, fall back to CSV on any Arrow issue.
    Returns the final written path.
    """
    df2 = df.copy()
    # Arrow is picky about some extension dtypes; normalize objects/categoricals to string
    for c in df2.columns:
        if isinstance(df2[c].dtype, pd.CategoricalDtype):
            df2[c] = df2[c].astype("string")
        elif pd.api.types.is_object_dtype(df2[c]):
            df2[c] = df2[c].astype("string")

    if fmt.lower() == "parquet":
        try:
            out_path = path_base.with_suffix(".parquet")
            df2.to_parquet(out_path, index=False)
            print(f"Saved Parquet -> {out_path}")
            return out_path
        except Exception as e:
            print(f"Parquet failed ({type(e).__name__}: {e}). Falling back to CSV.")
            out_path = path_base.with_suffix(".csv")
            df.to_csv(out_path, index=False)
            print(f"Saved CSV -> {out_path}")
            return out_path
    elif fmt.lower() == "csv":
        out_path = path_base.with_suffix(".csv")
        df.to_csv(out_path, index=False)
        print(f"Saved CSV -> {out_path}")
        return out_path
    else:
        raise ValueError("SAVE_FMT must be 'parquet' or 'csv'")

def _standardize_wave(df_in: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """
    - Drop wave prefix (k_/l_/n_) from column names
    - Build unified anxiety_raw: use 'mhgad' if present, else 'mhealthtyp1' if present
    - Add wave + wave_num
    - Ensure the standardized 21 columns are present (create missing as NA)
    - Return only STD_COLS + ['wave','wave_num'] in that order
    """
    df = df_in.copy()

    # shape before
    before_shape = df.shape

    # 1) Strip wave prefix on any column that has it
    pref = f"{prefix}_"
    rename_map = {c: c[len(pref):] for c in df.columns if c.startswith(pref)}
    df = df.rename(columns=rename_map)

    # 2) Build unified anxiety_raw
    if "mhgad" in df.columns:
        df["anxiety_raw"] = df["mhgad"]
    elif "mhealthtyp1" in df.columns:
        df["anxiety_raw"] = df["mhealthtyp1"]
    # If neither exists, will be added as NA in step 4

    # 3) Add wave indicators
    df["wave"] = prefix.upper()
    df["wave_num"] = WAVE_NUM[prefix]

    # 4) Ensure all STD_COLS exist; create missing as NA
    for c in STD_COLS:
        if c not in df.columns:
            df[c] = pd.NA

    # 5) Order columns: 21 standardized + wave + wave_num (23 total)
    df = df[STD_COLS + ["wave", "wave_num"]]

    after_shape = df.shape
    print(f"{prefix.upper()} -> BEFORE: {before_shape} | AFTER (standardized): {after_shape}")
    return df

# ---------- Run for K, L, N ----------
print("=== Loading saved per-wave columns-of-interest and standardizing ===")
k_path = _find_input_file("k")
l_path = _find_input_file("l")
n_path = _find_input_file("n")

k_in = _load_df(k_path)
l_in = _load_df(l_path)
n_in = _load_df(n_path)

k_std = _standardize_wave(k_in, "k")
l_std = _standardize_wave(l_in, "l")
n_std = _standardize_wave(n_in, "n")

# Optional: preview a few rows (comment out if you don't want any display)
print("\nK (first 10 rows):")
display(k_std.head(10))
print("\nL (first 10 rows):")
display(l_std.head(10))
print("\nN (first 10 rows):")
display(n_std.head(10))

# ---------- Save standardized per-wave frames ----------
_safe_to_parquet_or_csv(k_std, OUT_DIR / "ukhls_k_standardized", SAVE_FMT)
_safe_to_parquet_or_csv(l_std, OUT_DIR / "ukhls_l_standardized", SAVE_FMT)
_safe_to_parquet_or_csv(n_std, OUT_DIR / "ukhls_n_standardized", SAVE_FMT)

print("\nDone. Standardized files saved in:", OUT_DIR)
# ---- Quick sanity previews (10 rows each) from SAVED files only ----
k_sel = _load_saved("k")    


=== Loading saved per-wave columns-of-interest and standardizing ===
K -> BEFORE: (32008, 21) | AFTER (standardized): (32008, 23)
L -> BEFORE: (29271, 21) | AFTER (standardized): (29271, 23)
N -> BEFORE: (35471, 21) | AFTER (standardized): (35471, 23)

K (first 10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,68006127.0,68006127.0,68020564.0,0.0,1.703009,2019.0,49.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,26.41,16.0,0.0,K,11
1,68020564.0,68006127.0,68006127.0,0.0,0.000000,2019.0,48.0,1.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,1565.670044,34.68,12.0,0.0,K,11
2,68008847.0,68008847.0,-8.0,0.0,0.794699,2019.0,61.0,2.0,1.0,1.0,...,-8.0,1.0,14.0,39.0,2134.000000,44.20,9.0,0.0,K,11
3,68009527.0,68009527.0,68061288.0,0.0,0.962017,2019.0,41.0,1.0,1.0,1.0,...,-8.0,1.0,15.0,39.0,2043.000000,60.48,16.0,2.0,K,11
4,68061288.0,68009527.0,68009527.0,0.0,0.000000,2019.0,33.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,149.250000,54.23,11.0,2.0,K,11
5,68010887.0,68068082.0,68068082.0,0.0,1.055802,2019.0,55.0,2.0,1.0,1.0,...,-8.0,1.0,25.0,32.0,1250.000000,57.28,9.0,0.0,K,11
6,68068082.0,68068082.0,68010887.0,0.0,0.000000,2019.0,58.0,1.0,1.0,1.0,...,-8.0,1.0,19.0,-8.0,690.859985,56.15,9.0,0.0,K,11
7,68014287.0,68014287.0,-8.0,0.0,0.000000,2019.0,49.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,715.869995,44.37,23.0,1.0,K,11
8,68020407.0,68020407.0,-8.0,0.0,0.833442,2019.0,82.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,1621.670044,18.07,13.0,0.0,K,11
9,68028575.0,68157166.0,68157166.0,0.0,1.029255,2019.0,28.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,56.71,6.0,2.0,K,11



L (first 10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,68008847.0,68008847.0,-8.0,0.0,0.739967,2020.0,62.0,2.0,1.0,1.0,...,-8.0,1.0,14.0,39.0,2288.000000,37.58,12.0,0.0,L,12
1,68009527.0,68034180.0,68061288.0,0.0,0.950116,2020.0,43.0,1.0,1.0,1.0,...,-8.0,1.0,15.0,36.5,2060.000000,56.37,11.0,2.0,L,12
2,68061288.0,68034180.0,68009527.0,0.0,0.000000,2020.0,34.0,2.0,1.0,1.0,...,-8.0,2.0,24.0,10.0,474.000000,61.73,15.0,2.0,L,12
3,68010887.0,68010887.0,68068082.0,0.0,0.998976,2020.0,56.0,2.0,1.0,1.0,...,-8.0,1.0,25.0,32.0,1200.000000,51.64,11.0,0.0,L,12
4,68068082.0,68010887.0,68010887.0,0.0,0.000000,2020.0,59.0,1.0,1.0,1.0,...,-8.0,1.0,17.0,-8.0,3275.909912,56.15,10.0,0.0,L,12
5,68028575.0,68095380.0,68157166.0,0.0,1.031810,2020.0,28.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,2.0,L,12
6,68157166.0,68095380.0,68028575.0,0.0,0.000000,2020.0,35.0,1.0,1.0,1.0,...,-8.0,1.0,8.0,38.0,2974.000000,57.47,7.0,2.0,L,12
7,68029927.0,68029939.0,68029931.0,0.0,0.000000,2020.0,48.0,2.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,89.699997,53.79,9.0,0.0,L,12
8,68029939.0,68029939.0,-8.0,0.0,0.000000,2020.0,16.0,1.0,1.0,1.0,...,-8.0,-8.0,-8.0,-8.0,0.000000,57.76,8.0,0.0,L,12
9,68149808.0,68029939.0,-8.0,0.0,0.000000,2020.0,23.0,2.0,1.0,1.0,...,-8.0,1.0,7.0,30.0,865.000000,56.15,8.0,0.0,L,12



N (first 10 rows):


,pidp,hrpid,ppid,ind5mus_xw,indinui_lw,intdaty_dv,age_dv,sex_dv,gor_dv,urban_dv,...,anxiety_raw,jbft_dv,jbnssec_dv,jbhrs,fimnnet_dv,sf12pcs_dv,scghq1_dv,nchild_dv,wave,wave_num
0,22445.0,276841780.0,277059298.0,0.0,0.0,2022.0,37.0,2.0,8.0,1.0,...,2.0,1.0,2.0,28.0,1857.079956,62.83,24.0,2.0,N,14
1,29925.0,622866606.0,-8.0,0.0,0.0,2022.0,45.0,2.0,7.0,1.0,...,2.0,2.0,14.0,29.0,2378.750000,65.47,23.0,2.0,N,14
2,76165.0,141045780.0,142378492.0,0.0,0.0,2022.0,39.0,2.0,5.0,1.0,...,2.0,1.0,11.0,35.0,3206.000000,57.20,12.0,2.0,N,14
3,280165.0,783876922.0,756200970.0,0.0,0.0,2022.0,43.0,2.0,8.0,2.0,...,2.0,-8.0,-8.0,-8.0,94.470001,58.55,16.0,1.0,N,14
4,469205.0,414412580.0,-8.0,0.0,0.0,2022.0,32.0,2.0,4.0,1.0,...,2.0,2.0,25.0,16.0,2056.080078,49.93,16.0,2.0,N,14
5,599765.0,209943344.0,210167702.0,0.0,0.0,2022.0,35.0,2.0,5.0,1.0,...,2.0,1.0,2.0,37.0,2839.649902,56.15,6.0,1.0,N,14
6,732365.0,732365.0,-8.0,0.0,0.0,2022.0,37.0,1.0,2.0,1.0,...,1.0,-8.0,-8.0,-8.0,838.500000,45.49,35.0,0.0,N,14
7,1587125.0,1587125.0,-8.0,0.0,0.0,2022.0,56.0,2.0,1.0,1.0,...,2.0,1.0,7.0,37.0,2290.000000,37.37,15.0,0.0,N,14
8,2888645.0,2888645.0,-8.0,0.0,0.0,2022.0,33.0,2.0,11.0,1.0,...,2.0,1.0,2.0,38.0,2300.000000,56.15,6.0,0.0,N,14
9,3424485.0,103550562.0,-8.0,0.0,0.0,2022.0,86.0,2.0,6.0,2.0,...,2.0,-8.0,-8.0,-8.0,1092.750000,33.23,10.0,0.0,N,14


Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_k_standardized.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_l_standardized.parquet
Saved Parquet -> C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed\ukhls_n_standardized.parquet

Done. Standardized files saved in: C:\Users\User\Documents\Github\Gambling_MentalHealth_MSc\data\processed


In [21]:
from pathlib import Path
import pandas as pd

IN_DIR = PROJECT_ROOT / "data" / "interim"   # same as OUT_DIR above

def _load_wave(prefix: str, prefer_parquet=True):
    pqt = IN_DIR / f"ukhls_{prefix}_columns_of_interest.parquet"
    csv = IN_DIR / f"ukhls_{prefix}_columns_of_interest.csv"
    if prefer_parquet and pqt.exists():
        return pd.read_parquet(pqt)
    elif csv.exists():
        return pd.read_csv(csv)
    else:
        raise FileNotFoundError(f"No saved file found for wave '{prefix}' in {IN_DIR}")

k_sel = _load_wave("k")
l_sel = _load_wave("l")
n_sel = _load_wave("n")



In [5]:
# Cell 2b: Add unprefixed 'pidp' if missing so each wave has 21 cols
# Curated "columns of interest" you specified + 'pidp' to make 21 each

k_cols_interest = [
    "pidp",
    "k_hrpid","k_ppid","k_ind5mus_xw","k_indinui_lw",
    "k_intdaty_dv","k_age_dv","k_sex_dv",
    "k_gor_dv","k_urban_dv","k_hhtype_dv","k_hhsize",
    "k_ethn_dv","k_mhealthtyp1",
    "k_jbft_dv","k_jbnssec_dv","k_jbhrs",
    "k_fimnnet_dv","k_sf12pcs_dv","k_scghq1_dv","k_nchild_dv",
]

l_cols_interest = [
    "pidp",
    "l_hrpid","l_ppid","l_ind5mus_xw","l_indinui_lw",
    "l_intdaty_dv","l_age_dv","l_sex_dv",
    "l_gor_dv","l_urban_dv","l_hhtype_dv","l_hhsize",
    "l_ethn_dv","l_mhealthtyp1",
    "l_jbft_dv","l_jbnssec_dv","l_jbhrs",
    "l_fimnnet_dv","l_sf12pcs_dv","l_scghq1_dv","l_nchild_dv",
]

# For N you wanted anxiety included (n_mhgad) instead of origadd
n_cols_interest = [
    "pidp",
    "n_hrpid","n_ppid","n_ind5mus_xw","n_indinui_lw",
    "n_intdaty_dv","n_age_dv","n_sex_dv",
    "n_gor_dv","n_urban_dv","n_hhtype_dv","n_hhsize",
    "n_ethn_dv","n_mhgad",
    "n_jbft_dv","n_jbnssec_dv","n_jbhrs",
    "n_fimnnet_dv","n_sf12pcs_dv","n_scghq1_dv","n_nchild_dv",
]
# Ensure 'pidp' is included and cap to 21 columns
import pandas as pd

def ensure_21_with_pidp(prefix: str, cols: list[str]) -> list[str]:
    all_cols = pd.read_csv(DATA_INTERIM / f"ukhls_{prefix}_all_columns.csv")["column"].astype(str).tolist()

    # Add 'pidp' (unprefixed) if available and not already selected
    if "pidp" in all_cols and "pidp" not in cols:
        cols = ["pidp"] + cols  # put key first

    # cap to 21 in case we overshoot
    cols = cols[:21]

    # persist and report
    out = DATA_INTERIM / f"ukhls_{prefix}_columns_of_interest.csv"
    pd.Series(cols, name="column").to_frame().to_csv(out, index=False)
    print(f"{prefix.upper()} -> {len(cols)} columns")
    print(cols)
    print("-" * 120)
    return cols

k_cols_interest = ensure_21_with_pidp("k", k_cols_interest)
l_cols_interest = ensure_21_with_pidp("l", l_cols_interest)
n_cols_interest = ensure_21_with_pidp("n", n_cols_interest)


K -> 21 columns
['pidp', 'k_hrpid', 'k_ppid', 'k_ind5mus_xw', 'k_indinui_lw', 'k_intdaty_dv', 'k_age_dv', 'k_sex_dv', 'k_gor_dv', 'k_urban_dv', 'k_hhtype_dv', 'k_hhsize', 'k_ethn_dv', 'k_mhealthtyp1', 'k_jbft_dv', 'k_jbnssec_dv', 'k_jbhrs', 'k_fimnnet_dv', 'k_sf12pcs_dv', 'k_scghq1_dv', 'k_nchild_dv']
------------------------------------------------------------------------------------------------------------------------
L -> 21 columns
['pidp', 'l_hrpid', 'l_ppid', 'l_ind5mus_xw', 'l_indinui_lw', 'l_intdaty_dv', 'l_age_dv', 'l_sex_dv', 'l_gor_dv', 'l_urban_dv', 'l_hhtype_dv', 'l_hhsize', 'l_ethn_dv', 'l_mhealthtyp1', 'l_jbft_dv', 'l_jbnssec_dv', 'l_jbhrs', 'l_fimnnet_dv', 'l_sf12pcs_dv', 'l_scghq1_dv', 'l_nchild_dv']
------------------------------------------------------------------------------------------------------------------------
N -> 21 columns
['pidp', 'n_hrpid', 'n_ppid', 'n_ind5mus_xw', 'n_indinui_lw', 'n_intdaty_dv', 'n_age_dv', 'n_sex_dv', 'n_gor_dv', 'n_urban_dv', 'n_hhtyp

In [6]:
# Cell: Inspect labels for the 21 selected columns using wave prefix (pfx)
import pandas as pd
import pyreadstat
from pathlib import Path

# Map your selections by prefix
cols_interest_by_pfx = {
    "k": k_cols_interest,
    "l": l_cols_interest,
    "n": n_cols_interest,
}

def preview_selected_labels(pfx: str, save_csv: Path | None = None) -> pd.DataFrame:
    """
    Show (and optionally save) SPSS variable labels for the 21 selected columns in wave `pfx`.
    pfx: one of 'k', 'l', 'n'
    save_csv: optional Path to write a CSV (leave as None to not save)
    """
    assert pfx in ("k", "l", "n"), "pfx must be 'k', 'l', or 'n'"
    path = files[pfx]
    selected = cols_interest_by_pfx[pfx]

    # read only metadata
    _, meta = pyreadstat.read_sav(path, row_limit=1)
    name2label = dict(zip(meta.column_names, meta.column_labels))

    # check presence
    missing = [c for c in selected if c not in name2label]
    if missing:
        print(f"⚠️ Missing in {pfx.upper()} file: {missing}")

    rows = [{"variable": c, "label": name2label.get(c, "(no label)")} for c in selected]
    out = pd.DataFrame(rows)

    # nice ordering: keep your selection order
    out["order"] = range(len(out))
    out = out.sort_values("order").drop(columns="order").reset_index(drop=True)

    # display + optional save
    display(out)
    if save_csv is not None:
        out.to_csv(save_csv, index=False)
        print(f"Saved labels to: {save_csv}")

    return out

# --- Use it (no saving) ---
preview_selected_labels("k")
preview_selected_labels("l")
preview_selected_labels("n")

# If you ever want a file, pass a path (optional):
# preview_selected_labels("k", save_csv=Path("data/interim/ukhls_k_selected_labels.csv"))


,variable,label
0,pidp,Cross-wave person identifier (public release)
1,k_hrpid,Household reference person: PIDP
2,k_ppid,partner's person identifier: PIDP
3,k_ind5mus_xw,cross-sectional extra 5 minute interview perso...
4,k_indinui_lw,combined UKHLS+BHPS+IEMB longitudinal adult ma...
5,k_intdaty_dv,"Interview date: Year, derived"
6,k_age_dv,"Age, derived from dob_dv and intdat_dv"
7,k_sex_dv,"Sex, derived"
8,k_gor_dv,Government Office Region
9,k_urban_dv,"Urban or rural area, derived"


,variable,label
0,pidp,Cross-wave person identifier (public release)
1,l_hrpid,Household reference person: PIDP
2,l_ppid,partner's person identifier: PIDP
3,l_ind5mus_xw,cross-sectional extra 5 minute interview perso...
4,l_indinui_lw,combined UKHLS+BHPS+IEMB longitudinal adult ma...
5,l_intdaty_dv,"Interview date: Year, derived"
6,l_age_dv,"Age, derived from dob_dv and intdat_dv"
7,l_sex_dv,"Sex, derived"
8,l_gor_dv,Government Office Region
9,l_urban_dv,"Urban or rural area, derived"


,variable,label
0,pidp,Cross-wave person identifier (public release)
1,n_hrpid,Household reference person: PIDP
2,n_ppid,partner's person identifier: PIDP
3,n_ind5mus_xw,cross-sectional extra 5 minute interview perso...
4,n_indinui_lw,combined UKHLS+BHPS+IEMB longitudinal adult ma...
5,n_intdaty_dv,"Interview date: Year, derived"
6,n_age_dv,"Age, derived from dob_dv and intdat_dv"
7,n_sex_dv,"Sex, derived"
8,n_gor_dv,Government Office Region
9,n_urban_dv,"Urban or rural area, derived"


,variable,label
0,pidp,Cross-wave person identifier (public release)
1,n_hrpid,Household reference person: PIDP
2,n_ppid,partner's person identifier: PIDP
3,n_ind5mus_xw,cross-sectional extra 5 minute interview perso...
4,n_indinui_lw,combined UKHLS+BHPS+IEMB longitudinal adult ma...
5,n_intdaty_dv,"Interview date: Year, derived"
6,n_age_dv,"Age, derived from dob_dv and intdat_dv"
7,n_sex_dv,"Sex, derived"
8,n_gor_dv,Government Office Region
9,n_urban_dv,"Urban or rural area, derived"


In [7]:
MISS_LABELS = {-9:"missing", -8:"inapplicable", -7:"proxy/partial", -2:"refusal", -1:"don't know"}
MISS_CODES  = set(MISS_LABELS)

def _special_counts_numeric(s):
    v = pd.to_numeric(s, errors="coerce")
    return {MISS_LABELS[k]: int((v == k).sum()) for k in MISS_CODES}

def show_wave_diags_numeric(raw, mr, wave_name, sample_cols=("urban_dv","ppid","ethn_dv","jbhrs","anxiety_raw")):
    import pandas as pd
    print(f"\n=== {wave_name} diagnostics (numeric-aware) ===")
    missflags = [c for c in mr.columns if c.endswith("_missflag")]
    print(f"- missflag columns present: {len(missflags)}")

    # Which raw columns actually contained special negatives?
    had_negs = []
    for c in raw.columns:
        v = pd.to_numeric(raw[c], errors="coerce")
        if v.isin(MISS_CODES).any():
            had_negs.append(c)
    print(f"- raw columns with special negatives: {had_negs or 'None found'}")

    # Sample columns
    for c in sample_cols:
        if c in raw.columns:
            counts = _special_counts_numeric(raw[c])
            if any(counts.values()):
                print(f"  • {c}: raw special negatives -> {counts}")
            else:
                print(f"  • {c}: raw special negatives -> none")
            mf = f"{c}_missflag"
            if mf in mr.columns:
                print("    ", mf, "value_counts:", mr[mf].value_counts(dropna=False).to_dict())
            else:
                print("    ", mf, "absent (no special codes detected -> no flag).")
        else:
            print(f"  • {c}: not in this wave")

# Run after building k_raw/k_mr, l_raw/l_mr, n_raw/n_mr
show_wave_diags_numeric(k_raw, k_mr, "K")
show_wave_diags_numeric(l_raw, l_mr, "L")
show_wave_diags_numeric(n_raw, n_mr, "N")


NameError: name 'k_raw' is not defined

In [30]:
for col in [c for c in k_mr.columns if c.endswith("_missflag")]:
    # ensure "-8" got labeled somewhere if present in the raw
    base = col.replace("_missflag","")
    if (k_raw[base].astype(str).str.strip() == "-8").any() or (k_raw[base] == -8).any():
        vc = k_mr[col].value_counts(dropna=False)
        print(base, "-> has -8 label?", "inapplicable" in vc.index.tolist(), vc.to_dict())


mhealthtyp1 -> has -8 label? True {'inapplicable': 31487, 'proxy/partial': 464, 'observed': 56, 'refusal': 1}
jbhrs -> has -8 label? True {'inapplicable': 16589, 'observed': 15095, "don't know": 207, 'refusal': 117}
ppid -> has -8 label? True {'observed': 19938, 'inapplicable': 12070}
jbft_dv -> has -8 label? True {'observed': 17086, 'inapplicable': 13834, 'missing': 624, 'proxy/partial': 464}
jbnssec_dv -> has -8 label? True {'observed': 16672, 'inapplicable': 14257, 'missing': 1079}
scghq1_dv -> has -8 label? True {'observed': 30341, 'inapplicable': 1147, 'proxy/partial': 464, 'missing': 56}
sf12pcs_dv -> has -8 label? True {'observed': 30028, 'inapplicable': 1342, 'proxy/partial': 464, 'missing': 174}
anxiety_raw -> has -8 label? True {'inapplicable': 31487, 'proxy/partial': 464, 'observed': 56, 'refusal': 1}


In [1]:
# === Build a balanced panel on common pidp, then merge K/L/N ===
import pandas as pd

# Define k_std, l_std, and n_std by standardizing the raw data
k_std = read_and_standardize_raw("k", k_cols_interest)
l_std = read_and_standardize_raw("l", l_cols_interest)
n_std = read_and_standardize_raw("n", n_cols_interest)

def uniq_pidp(df):
    return df["pidp"].nunique()

print("Shapes BEFORE balance ->",
      "K:", k_std.shape, "L:", l_std.shape, "N:", n_std.shape)
print("Unique pidp BEFORE ->",
      "K:", uniq_pidp(k_std), "L:", uniq_pidp(l_std), "N:", uniq_pidp(n_std))

# 1) Identify people present in all three waves
pidp_k = set(k_std["pidp"].dropna())
pidp_l = set(l_std["pidp"].dropna())
pidp_n = set(n_std["pidp"].dropna())
common_pidp = pidp_k & pidp_l & pidp_n
print(f"Balanced panel size (unique pidp in all three): {len(common_pidp):,}")

# 2) Filter to balanced samples
k_bal = k_std[k_std["pidp"].isin(common_pidp)].reset_index(drop=True)
l_bal = l_std[l_std["pidp"].isin(common_pidp)].reset_index(drop=True)
n_bal = n_std[n_std["pidp"].isin(common_pidp)].reset_index(drop=True)

# 3) Make sure columns are identically ordered
cols = k_bal.columns.tolist()
l_bal = l_bal.reindex(columns=cols)
n_bal = n_bal.reindex(columns=cols)

# 4) Drop columns that are all-NA in all three balanced waves
all_na_cols = [
    c for c in cols
    if k_bal[c].isna().all() and l_bal[c].isna().all() and n_bal[c].isna().all()
]
if all_na_cols:
    k_bal = k_bal.drop(columns=all_na_cols)
    l_bal = l_bal.drop(columns=all_na_cols)
    n_bal = n_bal.drop(columns=all_na_cols)
    print("Dropped all-NA columns across K/L/N:", all_na_cols)

# 5) Concatenate into long balanced panel
long_balanced = (
    pd.concat([k_bal, l_bal, n_bal], ignore_index=True, copy=False)
      .sort_values(["pidp", "wave_num"])
      .reset_index(drop=True)
)

# 6) Report shapes AFTER balance
print("Shapes AFTER balance ->",
      "K:", k_bal.shape, "L:", l_bal.shape, "N:", n_bal.shape,
      "STACKED:", long_balanced.shape)

print("Unique pidp AFTER ->",
      "K:", k_bal["pidp"].nunique(), "L:", l_bal["pidp"].nunique(), "N:", n_bal["pidp"].nunique())
print(long_balanced.groupby("wave")["pidp"].nunique())

# 7) Duplicate check on pidp + wave_num
def dup_count(df, keys=("pidp", "wave_num")):
    return int(df.duplicated(list(keys)).sum())

print("Duplicate rows on ['pidp','wave_num'] ->",
      "K:", dup_count(k_bal),
      "L:", dup_count(l_bal),
      "N:", dup_count(n_bal),
      "STACKED:", dup_count(long_balanced))

# 8) Peek at the merged long table
display(long_balanced.head(10))


NameError: name 'read_and_standardize_raw' is not defined

In [2]:
def check_clean_vs_flag(df, cols):
    for c in cols:
        flag = df[f"{c}_missflag"]
        clean = df[f"{c}_clean"]
        # 1) any labeled as special negative must be NA in _clean
        assert clean[flag != "observed"].isna().all(), f"{c}: some non-observed not NA in _clean"
        # 2) any labeled observed should equal raw (ignoring float/int tiny diffs)
        same = (clean[flag == "observed"].fillna(pd.NA) == df[c][flag == "observed"].fillna(pd.NA))
        assert same.all(), f"{c}: observed values changed in _clean"
    print("All clean↔flag invariants hold.")

for df, cleaned, name in [(k_mr, k_cleaned, "K"), (l_mr, l_cleaned, "L"), (n_mr, n_cleaned, "N")]:
    print(f"\n{name}:")
    check_clean_vs_flag(df, cleaned)


NameError: name 'k_mr' is not defined